In [1]:
import pandas as pd
import numpy as np
import time
from zipfile import ZipFile
import time
import matplotlib as mpl
from xlsxwriter.utility import xl_rowcol_to_cell
from pretty_html_table import build_table
from datetime import date,datetime,timedelta
from babel.numbers import format_currency
from IPython.core.display import HTML
from pyxlsb import open_workbook
from zipfile import ZipFile
import shutil
import os
import glob

##### Email Imports

In [2]:
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib

In [3]:
from datetime import date
day = date.today()
print("Today date is: ", day)
tday= day
#today=input()

Today date is:  2023-10-26


In [4]:
t_day=pd.to_datetime(tday,format="%Y-%m-%d")

In [5]:
t_day

Timestamp('2023-10-26 00:00:00')

In [6]:
pd.Series.str.rereplace = lambda self, pat, repl, n=-1, case=None, flags=0: self.replace(pat, repl, n, case, flags, regex=True)

In [7]:
pd.Series.str.recontains = lambda self, pat, case=True, flags=0, na=None: self.contains(pat, case, flags, na, regex=True)

###### Master

In [8]:
master_path="D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Aster Pharmacy Raw data\Masters\\"

In [9]:
master_path

'D:\\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\\Aster Pharmacy Raw data\\Masters\\'

In [10]:
gof_pro=pd.read_csv(master_path+'Gof Pro master.csv',
                  skiprows =3 ,low_memory=False)
gof_pro.drop(gof_pro.index[0],inplace=True)
dyn_pro1=pd.read_csv(master_path+'D365 pro Master 1.csv',low_memory=False)
dyn_pro2=pd.read_csv(master_path+'D365 pro Master 2.csv',low_memory=False)
uniq_master=pd.read_csv(master_path+'Item Unique master.csv',low_memory=False)
sd_details=pd.read_csv(master_path+'Store Details.csv',low_memory=False)
ctg_pro=pd.read_csv(master_path+'CTG.csv',low_memory=False)
mdm_pro=pd.read_csv(master_path+'One_MDM_Master.csv',low_memory=False)
dis_details=pd.read_excel(master_path+'Dist_details_master.xlsx')
st_store_map=pd.read_excel(master_path+'Store Mapper For Stock in Transit.xlsx')

In [11]:
print(dyn_pro1.shape,dyn_pro2.shape)

(89026, 41) (336, 38)


In [12]:
dyn_pro=pd.concat([dyn_pro1,dyn_pro2])
dyn_pro.reset_index(inplace=True)
dyn_pro.drop("index",axis=1,inplace=True)

In [13]:
dyn_pro['Item number'].count()==dyn_pro1['Item number'].count()+dyn_pro2['Item number'].count()

True

In [14]:
print("Gof Item master:",gof_pro.shape)
print("Dyn Item master(1):",dyn_pro1.shape)
print("Dyn Item master(2):",dyn_pro2.shape)
print("Dyn Item master:",dyn_pro.shape)
print("Unique Item master:",uniq_master.shape)
print("Stores Details:",sd_details.shape)
print("CTG Master:",ctg_pro.shape)
print("MDM Master:",mdm_pro.shape)

Gof Item master: (90225, 18)
Dyn Item master(1): (89026, 41)
Dyn Item master(2): (336, 38)
Dyn Item master: (89362, 44)
Unique Item master: (90225, 40)
Stores Details: (274, 58)
CTG Master: (26, 2)
MDM Master: (86327, 15)


In [15]:
sd_details.rename(columns={'Store Mapping As Per grofula':"Outlet Name",
 'Status':"Gof_status",'D 365 Store Id':"Storeid"}, inplace=True)

#### Master Modifications

In [16]:
gof_pro.rename(columns={'Generic':"CTG"},inplace=True)
gof_pro['CTG']=gof_pro['CTG'].str.upper()
gof_pro.head(1)

,Item Code,Item Name,Manufacturer Name,CTG,Therapeutic,BOM,Medical Item Type,KOM,Generic Category,Generic Name,Main Supplier Name,I_U,Strength,Choose Tax,Basic Cost,Selling Price,MRP,Purchase Price
1,1.0,PANTOSEC 40 MG TABLET 10S,CIPLA GENERIC (AAGAM ),PHARMA,NaN,NaN,1-Scheduled,FMCG,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,10.0,.,GST 12%,0.0,0.00,0.00,0.0


In [17]:
dyn_pro.rename(columns={'Item group':"CTG",'Item number':"Item Code"},inplace=True)
dyn_pro['CTG']=dyn_pro['CTG'].str.upper()
dyn_pro.head(1)

,Item Code,Product name,Search name,Item Reference,Composition,Product type,Product subtype,Product dimension groups,Product lifecycle state,Chronic Drug,...,MDM Clean 1,IS Purchasse (B),One MDM Code,MDM Chcek,Is Sale Blocked,Created Date,CHAR =10,Mfg Code2,Reference,MDM CHECK
0,P-0000002,CIPZEN D 50MG/10MG TAB 10S,CIPZEN-D TABLET,3-A-AY42-HW-BB17-XX09-X-00001,DICLOFENAC SODIUM + SERRATIOPEPTIDASE,Item,Product,NaN,NaN,No,...,3-A-AY42-HW-BB17-XX09-X-00001,No,3-A-AY42-HW-BB17-XX09-X-00001,False,No,01-Apr-22,False,NaN,NaN,NaN


In [18]:
sd_details.columns

Index(['S. No.', 'Store Code', 'Store Type', 'HDH Store Code', 'HPD BU2 ID',
       'HPD TNG', 'Get Well', 'SreeJi', 'Vasu agency', 'Venkat Sai',
       'HPD Kerala', 'PO Store Name', 'Store NameAs per Branding ',
       'Outlet Name', 'GOF Store Id', 'Launch Date', 'Email ID', 'Short Name',
       'Build-up Area in Sft', 'Phone Number', 'Ops Exexutive',
       'Cluster Mail Id', 'Executive Phone Number', 'Area Manager',
       'Sub Region', 'State', 'City', 'City 2', 'Gof_status', 'Manager',
       'Cluster', 'Any Desk ID', 'Rental', 'FF Target', 'Address', 'Zip Code',
       'Region', 'HOO', 'Outlet Format', 'Tenure Days', 'Tenure Month',
       'Tenure Years', 'Tenure', 'Launch Month', 'Launch Year', 'Quarter',
       'Financial Year', 'Securty Deposit', 'Rent Start Date', 'Store System',
       'Storeid', 'Store Name As per D 365', 'Retail Channel Id', 'Party ID',
       'Fin And Qtr', 'Location Series', 'City G', 'Vendor Code'],
      dtype='object')

###### Purchase Line  and Sales Data Import for dynamics & gofrugal

In [19]:
raw_data_path="D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Historical Raw Data\\"

In [20]:
raw_data_path

'D:\\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\\Historical Raw Data\\'

### Dynamics

###### Purch Lines

In [21]:
# specifying the path to csv files
d365_purch_path =raw_data_path+"D 365 Raw data\Purch line Item wise"
# csv files in the path
d365_purch_files = glob.glob(d365_purch_path + "/*.csv")

dyn_pr_lines= pd.DataFrame()
dyn_pr_lines_content = []

for filename in d365_purch_files:
    
    # reading content of csv file
    # content.append(filename)
    d365_purch_rd = pd.read_csv(filename, index_col=None,low_memory=False)
    d365_purch_rd["Qty"]=d365_purch_rd["Qty"].astype(str)
    dyn_pr_lines_content.append(d365_purch_rd)
    
    
dyn_pr_lines = pd.concat(dyn_pr_lines_content)

In [22]:
dyn_pr_lines.reset_index(inplace=True)
dyn_pr_lines.drop("index",axis=1,inplace=True)
print(dyn_pr_lines.shape)
dyn_pr_lines.tail(2)

(1315902, 41)


,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,TaxAmt,CreatedUser,Textbox235,Textbox205,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214
1315900,TS-HO,Telangana Head Office,PO-372153,23-10-2023,ARPL-233828,23-10-2023,132,23-10-2023,NaN,NaN,...,2448.0,shridhar,1348341.0,51304782.19,57863627.1,1996328.42,59859955.52,50813312.77,180828.86,7071594.97
1315901,TS-HO,Telangana Head Office,PO-372153,23-10-2023,ARPL-233828,23-10-2023,132,23-10-2023,NaN,NaN,...,459.0,shridhar,1348341.0,51304782.19,57863627.1,1996328.42,59859955.52,50813312.77,180828.86,7071594.97


###### Sales Data

In [23]:
d365_sales_path =raw_data_path+"D 365 Raw data\Sales Data"

d365_sales_files = glob.glob(d365_sales_path + "/*.csv")

dyn_s= pd.DataFrame()
dyn_s_content = []

for filename in d365_sales_files:
    
    
    d365_sales_rd = pd.read_csv(filename, index_col=None,low_memory=False)
    d365_sales_rd["Quantity"]=d365_sales_rd["Quantity"].astype(str)
    dyn_s_content.append(d365_sales_rd)

In [24]:
dyn_s = pd.concat(dyn_s_content)

In [25]:
dyn_s.reset_index(inplace=True)
dyn_s.drop("index",axis=1,inplace=True)
print(dyn_s.shape)
dyn_s.tail(2)

(2466104, 54)


,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox34,Textbox90,Textbox91,Textbox92,Textbox93,Textbox94,Textbox95,Textbox97,Textbox83,Textbox70
2466102,3201029,KL - Thiruvamkulam,26-10-2023,0:37:23,Sales,I320102912202481,Thiruvankulam,AD-000002,"Buy Pharma get 10%,Buy FMCG get 5%,Buy Baby ge...",NaN,...,49393440.62,2722404.27,2722356.76,111.75,0,7.700740e+06,507168.78,8.207909e+06,10674123.61,10674123.61
2466103,3201029,KL - Thiruvamkulam,26-10-2023,0:37:23,Sales,I320102912202481,Thiruvankulam,AD-000002,"Buy Pharma get 10%,Buy FMCG get 5%,Buy Baby ge...",NaN,...,49393440.62,2722404.27,2722356.76,111.75,0,7.700740e+06,507168.78,8.207909e+06,10674123.61,10674123.61


### Gofrugal

###### Purch Lines

In [26]:
gof_purch_path =raw_data_path+"Gofrugal Raw Data\Purch line Item wise"
gof_purch_files = glob.glob(gof_purch_path + "/*.csv")
gof_pr_lines= pd.DataFrame()
gof_pr_lines_content = []

for filename in gof_purch_files:
 
    gof_purch_rd = pd.read_csv(filename, index_col=None,low_memory=False)
    gof_purch_rd["Qty"]=gof_purch_rd["Qty"].astype(str)
    gof_pr_lines_content.append(gof_purch_rd)
    
gof_pr_lines = pd.concat(gof_pr_lines_content)

In [27]:
gof_pr_lines.reset_index(inplace=True)
gof_pr_lines.drop("index",axis=1,inplace=True)
print(gof_pr_lines.shape)
gof_pr_lines.tail(2)

(805086, 74)


,OUTLET NAME,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,Dist. Name,Item Code,Item Name,Batch No.,Expiry Dt.,...,SCHEME DISCOUNT PER QTY,TO DIFF CWT PER QTY,Units,Qty(Tablets),Free Qty(Tablets),Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling
805084,TG - Yousufguda Check Post,MR1479,16-10-2023,HPDTL-23-24859,13-10-2023,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED TL,991240,CLOSE UP MULTIVITAMIN TOOTHPASTE 80G,,01-07-2025,...,0.0,0,1,2,0,0,0.0,0,90.48,120
805085,TG - Yousufguda Check Post,MR1479,16-10-2023,HPDTL-23-24859,13-10-2023,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED TL,991246,VASELINE LIGHT HYDRATE LOTION 100ML,,01-07-2025,...,0.0,0,1,2,0,0,0.0,0,154.06,198


###### Sales Data

In [28]:
gof_sales_path =raw_data_path+"Gofrugal Raw Data\Sales Data\MW"

gof_sales_files = glob.glob(gof_sales_path + "/*.csv")

gof_s= pd.DataFrame()
gof_s_content = []

for filename in gof_sales_files:
    
    
    gof_sales_rd = pd.read_csv(filename, index_col=None,low_memory=False)
    gof_sales_rd["Qty"]=gof_sales_rd["Qty"].astype(str)
    gof_s_content.append(gof_sales_rd)
    
    
gof_s = pd.concat(gof_s_content)

In [29]:
gof_s.reset_index(inplace=True)
gof_s.drop("index",axis=1,inplace=True)
print(gof_s.shape)
gof_s.tail(2)

(1899309, 30)


,Outlet Name,Bill No.,Bill Date,Item Code,Item Name,Phone1,Mobile Number & SMS alerts,Mobile2,Telephone,CUST GST NO,...,Net Sales (Without Vat),Net Cost (With Vat),VatAmt (Purchase),Net Cost (Without Vat),Margin On NS(WOT Vat),Margin On NS(With Vat),Margin%,Landing Cost,MRP,Item Rate
1899307,TG - Yousufguda Check Post,WL17744,16-10-2023,39,PARACIP 650MG TABLET 10S,,6366619466,,6366619466,,...,17.89,14.5,1.6,12.9,4.99,5.54,27.90,1.45,20.04,2
1899308,TG - Yousufguda Check Post,WL17744,16-10-2023,990225,REBALANZ ORS LEMON 200ML,,6366619466,,6366619466,,...,71.42,69.21,7.41,61.8,9.62,10.78,13.47,23.07,40,40


In [30]:
gof_s.head(3)

,Outlet Name,Bill No.,Bill Date,Item Code,Item Name,Phone1,Mobile Number & SMS alerts,Mobile2,Telephone,CUST GST NO,...,Net Sales (Without Vat),Net Cost (With Vat),VatAmt (Purchase),Net Cost (Without Vat),Margin On NS(WOT Vat),Margin On NS(With Vat),Margin%,Landing Cost,MRP,Item Rate
0,AP - Currency Nagar,CA1838,01-04-2023,110048,KOFLET LOZENGES JAR 200S,,9515370679,,9515370679,,...,8.57,7.38,0.78,6.6,1.97,2.22,22.99,1.23,1.6,1.6
1,AP - Currency Nagar,CA1838,01-04-2023,343022,GLIMESTAR PM 1,,9515370679,,9515370679,,...,152.66,142.8,15.2,127.6,25.06,28.18,16.42,7.14,95,9.5
2,AP - Currency Nagar,CA1838,01-04-2023,535251,VICKS MINT COUGH DROPS 20`S,,9515370679,,9515370679,,...,17.86,16.69,1.79,14.9,2.96,3.31,16.55,16.69,20,20


In [31]:
gof_dis=gof_pr_lines.loc[:,['Dist. Name']]
gof_dis.rename(columns={'Dist. Name':"VendName"},inplace=True)
print(gof_dis.shape)
gof_dis.head(2)

(805086, 1)


,VendName
0,PRUDHVI PHARMACEUTICALS
1,PRUDHVI PHARMACEUTICALS


In [32]:
dyn_dis=dyn_pr_lines.loc[:,['VendName']]
print(dyn_dis.shape)
dyn_dis.head(2)

(1315902, 1)


,VendName
0,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...
1,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...


In [33]:
dist_con=pd.concat([gof_dis,dyn_dis])

In [34]:
print(dist_con.shape)
dist_con.head(2)

(2120988, 1)


,VendName
0,PRUDHVI PHARMACEUTICALS
1,PRUDHVI PHARMACEUTICALS


In [35]:
dis_details.rename(columns={'Dist. Name':"VendName"},inplace=True)
dis_details.columns

Index(['VendName', 'Dist type', 'Old'], dtype='object')

In [36]:
dist_con=pd.merge(dist_con,dis_details[['Dist type','VendName',]], on='VendName', how='left')

In [37]:
dist_con.rename(columns={'Dist type':"Vendor Group"},inplace=True)
dist_con["Vendor Group"].fillna("No Data Found", inplace = True)
dist_con.head(1)

,VendName,Vendor Group
0,PRUDHVI PHARMACEUTICALS,Outside Purchase


In [38]:
no_data=dist_con.drop(dist_con[dist_con['Vendor Group']!='No Data Found'].index)
unq_m_dis=no_data.drop_duplicates(subset = "VendName")

In [39]:
dyn_pro.rename(columns={'Item number':"ItemId",'Item group':"CTG"},inplace=True)
dyn_pro['CTG']=dyn_pro['CTG'].str.upper()

In [40]:
dis_details.rename(columns={'Dist type':"Vendor Group"},inplace=True)
dis_details.columns

Index(['VendName', 'Vendor Group', 'Old'], dtype='object')

In [41]:
dyn_pr_lines=pd.merge(dyn_pr_lines,dis_details[['VendName','Vendor Group']],on='VendName',how='left')
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,CreatedUser,Textbox235,Textbox205,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,it-d365,333035.0,30313955.01,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,it-d365,333035.0,30313955.01,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor


In [42]:
dyn_pr_lines.rename(columns={'ItemId':"Item Code"},inplace=True)
dyn_pr_lines.head(1)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,CreatedUser,Textbox235,Textbox205,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,it-d365,333035.0,30313955.01,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor


In [43]:
dyn_pr_lines['Qty']=dyn_pr_lines['Qty'].str.replace(",","").astype(float)
dyn_pr_lines.loc[dyn_pr_lines['Qty'] < 0, 'Grn Type'] = 'Grn Returns'
dyn_pr_lines.loc[dyn_pr_lines['Qty'] >= 0, 'Grn Type'] = 'Grn'

In [44]:
dyn_pr_lines=dyn_pr_lines.drop(dyn_pr_lines[dyn_pr_lines['Grn Type']!='Grn'].index)

In [45]:
dyn_pr_lines.reset_index(drop=True,inplace=True)

In [46]:
print(dyn_pr_lines.shape)
dyn_pr_lines.tail(2)

(1134746, 43)


,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox235,Textbox205,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type
1134744,TS-HO,Telangana Head Office,PO-372153,23-10-2023,ARPL-233828,23-10-2023,132,23-10-2023,NaN,NaN,...,1348341.0,51304782.19,57863627.1,1996328.42,59859955.52,50813312.77,180828.86,7071594.97,Outside Purchase,Grn
1134745,TS-HO,Telangana Head Office,PO-372153,23-10-2023,ARPL-233828,23-10-2023,132,23-10-2023,NaN,NaN,...,1348341.0,51304782.19,57863627.1,1996328.42,59859955.52,50813312.77,180828.86,7071594.97,Outside Purchase,Grn


In [47]:
dyn_pr_lines.columns

Index(['Store', 'StoreName', 'PONO', 'PODate', 'GRNNo', 'GRNDate',
       'ProductReceiptNo', 'DocDate', 'InvoiceNo', 'InvoiceDate', 'VendorID',
       'VendName', 'Item Code', 'ItemName', 'Category', 'Batch', 'ExpiryDate',
       'Qty', 'PurchPrice', 'MRP', 'LandingCost', 'ItemAmount',
       'ItemNetAmount', 'Margin', 'MarginAmt', 'MRPValue', 'ItemBeforeTax',
       'SCHEMEDISCOUNT', 'FREIGHTCHARGES', 'HSNCode', 'TaxPercent', 'TaxAmt',
       'CreatedUser', 'Textbox235', 'Textbox205', 'Textbox206', 'Textbox208',
       'Textbox209', 'Textbox210', 'Textbox16', 'Textbox214', 'Vendor Group',
       'Grn Type'],
      dtype='object')

In [48]:
dyn_pr_lines=pd.merge(dyn_pr_lines,dyn_pro[['Item Code','Unit']],on='Item Code',how='left')
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox205,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type,Unit
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,30313955.01,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,ea
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,30313955.01,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,10's


In [49]:
dyn_pro.columns

Index(['Item Code', 'Product name', 'Search name', 'Item Reference',
       'Composition', 'Product type', 'Product subtype',
       'Product dimension groups', 'Product lifecycle state', 'Chronic Drug',
       'Is generic', 'Manufacturer', 'Mfg Code', 'Marketed By',
       'Is Prescription Required', 'Product Form', 'Schedule Type', 'BOM unit',
       'Dual Use Code', 'Item type', 'Packing group', 'Packing quantity',
       'Production group', 'Production type', 'Search name2', 'Tax rate type',
       'Vendor', 'Modified by', 'Unit', 'Created by', 'CTG',
       'Item model group', 'Created date and time', 'HSN codes', 'MDM Clean 1',
       'IS Purchasse (B)', 'One MDM Code', 'MDM Chcek ', 'Is Sale Blocked',
       'Created Date', 'CHAR =10', 'Mfg Code2', 'Reference', 'MDM CHECK'],
      dtype='object')

In [50]:
dyn_pr_lines=pd.merge(dyn_pr_lines,dyn_pro[['Item Code','CTG']],on='Item Code',how='left')
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox206,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type,Unit,CTG
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,ea,PHARMA
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,33675425.62,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,10's,PHARMA


In [51]:
dyn_pr_lines=pd.merge(dyn_pr_lines,ctg_pro[['CTG','CTG PRO']],on='CTG',how='left')
dyn_pr_lines['CTG']=dyn_pr_lines['CTG'].str.upper()

In [52]:
dyn_pr_lines["CTG"].fillna("NO DATA", inplace = True)
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA


In [53]:
dyn_pr_lines["CTG PRO"].fillna("NO DATA", inplace = True)
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA


In [54]:
gof_pr_lines.rename(columns={'Dist. Name':"VendName",'OUTLET NAME':"Outlet Name"},inplace=True)
#gof_pr_lines.columns

In [55]:
sd_details.rename(columns={'Store Mapping As Per grofula':"Outlet Name",
 'Status':"Gof_status",'D 365 Store Id':"Storeid"}, inplace=True)
#sd_details.columns

In [56]:
gof_pr_lines=pd.merge(gof_pr_lines,sd_details[['Outlet Name','Storeid']],on='Outlet Name',how='left')

In [57]:
gof_pr_lines=pd.merge(gof_pr_lines,dis_details[['VendName','Vendor Group']],on='VendName',how='left')
gof_pr_lines.head(2)

,Outlet Name,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Units,Qty(Tablets),Free Qty(Tablets),Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,10,100,0.0,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,1,3,0.0,0,0.0,0,118.05,175.8,3701001,Outside Purchase


In [58]:
gof_pro.columns

Index(['Item Code', 'Item Name', 'Manufacturer Name', 'CTG', 'Therapeutic',
       'BOM', 'Medical Item Type', 'KOM', 'Generic Category', 'Generic Name',
       'Main Supplier Name', 'I_U', 'Strength', 'Choose Tax', 'Basic Cost',
       'Selling Price', 'MRP', 'Purchase Price'],
      dtype='object')

In [59]:
gof_pr_lines=pd.merge(gof_pr_lines,gof_pro[['Item Code','I_U']],on='Item Code',how='left')
gof_pr_lines.head(5)

,Outlet Name,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Qty(Tablets),Free Qty(Tablets),Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,100,0.0,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase,10.0
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,3,0.0,0,0.0,0,118.05,175.8,3701001,Outside Purchase,1.0
2,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,144011,GELUSIL MPS LIQ (MF),2219314E,01-09-2024,...,2,0.0,0,0.0,0,186.82,278.26,3701001,Outside Purchase,1.0
3,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,157023,ONDEM 4MG TAB,22443283,01-10-2024,...,30,0.0,0,0.0,0,103.2,153.9,3701001,Outside Purchase,10.0
4,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,157025,ONDEM 8MG TAB,22443093,01-09-2024,...,30,0.0,0,0.0,0,233.1,347.31,3701001,Outside Purchase,10.0


In [60]:
gof_pr_lines.head(5)

,Outlet Name,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Qty(Tablets),Free Qty(Tablets),Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,100,0.0,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase,10.0
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,3,0.0,0,0.0,0,118.05,175.8,3701001,Outside Purchase,1.0
2,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,144011,GELUSIL MPS LIQ (MF),2219314E,01-09-2024,...,2,0.0,0,0.0,0,186.82,278.26,3701001,Outside Purchase,1.0
3,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,157023,ONDEM 4MG TAB,22443283,01-10-2024,...,30,0.0,0,0.0,0,103.2,153.9,3701001,Outside Purchase,10.0
4,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,157025,ONDEM 8MG TAB,22443093,01-09-2024,...,30,0.0,0,0.0,0,233.1,347.31,3701001,Outside Purchase,10.0


In [61]:
gof_pro.rename(columns={'Generic':"CTG"},inplace=True)
gof_pro['CTG']=gof_pro['CTG'].str.upper()
gof_pro.columns

Index(['Item Code', 'Item Name', 'Manufacturer Name', 'CTG', 'Therapeutic',
       'BOM', 'Medical Item Type', 'KOM', 'Generic Category', 'Generic Name',
       'Main Supplier Name', 'I_U', 'Strength', 'Choose Tax', 'Basic Cost',
       'Selling Price', 'MRP', 'Purchase Price'],
      dtype='object')

In [62]:
gof_pr_lines=pd.merge(gof_pr_lines,gof_pro[['Item Code','CTG']],on='Item Code',how='left')

In [63]:
gof_pr_lines.head(1)

,Outlet Name,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Free Qty(Tablets),Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U,CTG
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,0.0,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase,10.0,PHARMA


In [64]:
gof_pr_lines=pd.merge(gof_pr_lines,ctg_pro[['CTG','CTG PRO']],on='CTG',how='left')

In [65]:
gof_pr_lines["CTG"].fillna("NO DATA", inplace = True)
gof_pr_lines["CTG PRO"].fillna("NO DATA", inplace = True)
gof_pr_lines.head(1)

,Outlet Name,Purchase No.,Purchase Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U,CTG,CTG PRO
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase,10.0,PHARMA,PHARMA


In [66]:
gof_ctg=gof_pr_lines.loc[:,['CTG','CTG PRO','Item Code', 'Item Name']]

In [67]:
gof_ctg.head(2)

,CTG,CTG PRO,Item Code,Item Name
0,PHARMA,PHARMA,107009,AUGMENTIN 625 DUO
1,PHARMA,PHARMA,139008,BETNOVATE -C SKIN CR


In [68]:
dyn_ctg=dyn_pr_lines.loc[:,['CTG','CTG PRO','Item Code', 'ItemName']]

In [69]:
dyn_ctg.rename(columns={ 'ItemName':"Item Name"},inplace=True)

In [70]:
ctg_ch=pd.concat([gof_ctg,dyn_ctg])

In [71]:
ctg_ch.head(1)

,CTG,CTG PRO,Item Code,Item Name
0,PHARMA,PHARMA,107009,AUGMENTIN 625 DUO


In [72]:
ctg_unq=ctg_ch.drop_duplicates(subset = "CTG")
ctg_mis=ctg_unq.drop(ctg_unq[ctg_unq['CTG PRO']!='NO DATA'].index)

In [73]:
ctg_mis.head()

,CTG,CTG PRO,Item Code,Item Name


In [74]:
date=time.strftime("%d-%m-%Y")

In [75]:
missed_rd="D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Aster Pharmacy Raw data\Phyton RD Drive\Missing RD\\"

In [76]:
missing_report=missed_rd+"Purchlines Missing Data "+date +".xlsx"

In [77]:
missing_report

'D:\\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\\Aster Pharmacy Raw data\\Phyton RD Drive\\Missing RD\\Purchlines Missing Data 26-10-2023.xlsx'

In [78]:
unq_m_dis.head()

,VendName,Vendor Group


In [79]:
ctg_mis.head()

,CTG,CTG PRO,Item Code,Item Name


In [80]:
with pd.ExcelWriter(missing_report) as writer:
    ctg_mis.to_excel(writer,sheet_name='Missing',index=False)    
    unq_m_dis.to_excel(writer,sheet_name='Missing',startcol=5,index=False)

In [81]:
check=input()

In [82]:
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox208,Textbox209,Textbox210,Textbox16,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,7357297.2,41032722.83,29783740.7,43440.01,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA


In [83]:
dyn_pr_lines[['DD','MM','YYY']]=dyn_pr_lines['GRNDate'].str.split("-",expand=True)
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox16,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO,DD,MM,YYY
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,43440.01,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA,04,04,2023
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,43440.01,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA,04,04,2023


In [84]:
dyn_pr_lines['Purchase Date'] = dyn_pr_lines['MM'].map(str)+"/"+dyn_pr_lines['DD'].map(str)+"/"+dyn_pr_lines['YYY'].map(str)
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO,DD,MM,YYY,Purchase Date
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA,04,04,2023,04/04/2023
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA,04,04,2023,04/04/2023


In [85]:
dyn_pr_lines['Purchase Date'] = dyn_pr_lines['Purchase Date'].astype('datetime64[ns]')
dyn_pr_lines.head(2)

,Store,StoreName,PONO,PODate,GRNNo,GRNDate,ProductReceiptNo,DocDate,InvoiceNo,InvoiceDate,...,Textbox214,Vendor Group,Grn Type,Unit,CTG,CTG PRO,DD,MM,YYY,Purchase Date
0,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,3927745.96,Generic Distibutor,Grn,ea,PHARMA,PHARMA,04,04,2023,2023-04-04
1,2901001,KA - Dasarahalli,PO-217707,04-04-2023,ARPL-121316,04-04-2023,HPD-22-106705,28-03-2023,HPD-22-106705,28-03-2023,...,3927745.96,Generic Distibutor,Grn,10's,PHARMA,PHARMA,04,04,2023,2023-04-04


In [86]:
dyn_grn=dyn_pr_lines.loc[:,['Purchase Date','Store', 'StoreName','VendName','Item Code', 'ItemName','Vendor Group','CTG PRO','Qty','ItemNetAmount']]

In [87]:
dyn_grn.head(2)

,Purchase Date,Store,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount
0,2023-04-04,2901001,KA - Dasarahalli,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0001673,CREMAFFIN MIXED SF FRUIT 225ML,Generic Distibutor,PHARMA,1.0,181.91
1,2023-04-04,2901001,KA - Dasarahalli,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0011342,FIBROEASE 25 TABLET,Generic Distibutor,PHARMA,1.0,599.51


In [88]:
dyn_grn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1134746 entries, 0 to 1134745
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Purchase Date  1134746 non-null  datetime64[ns]
 1   Store          1134746 non-null  object        
 2   StoreName      1134746 non-null  object        
 3   VendName       1134746 non-null  object        
 4   Item Code      1134746 non-null  object        
 5   ItemName       1134746 non-null  object        
 6   Vendor Group   1134746 non-null  object        
 7   CTG PRO        1134746 non-null  object        
 8   Qty            1134746 non-null  float64       
 9   ItemNetAmount  1134746 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 95.2+ MB


In [89]:
gof_pr_lines.rename(columns={'Purchase Date':"Pur Date"},inplace=True)
gof_pr_lines.head(2)

,Outlet Name,Purchase No.,Pur Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Physical Qty,Discrepancy Qty,Spl Rate,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U,CTG,CTG PRO
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,0,0.0,0,"1,227.00","1,827.80",3701001,Outside Purchase,10.0,PHARMA,PHARMA
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,0,0.0,0,118.05,175.8,3701001,Outside Purchase,1.0,PHARMA,PHARMA


In [90]:
gof_pr_lines[['DD','MM','YYY']]=gof_pr_lines['Pur Date'].str.split("-",expand=True)
gof_pr_lines.head(2)

,Outlet Name,Purchase No.,Pur Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Purchase value(WOT),Total Selling,Storeid,Vendor Group,I_U,CTG,CTG PRO,DD,MM,YYY
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,"1,227.00","1,827.80",3701001,Outside Purchase,10.0,PHARMA,PHARMA,03,04,2023
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,118.05,175.8,3701001,Outside Purchase,1.0,PHARMA,PHARMA,03,04,2023


In [91]:
gof_pr_lines['Purchase Date'] = gof_pr_lines['MM'].map(str)+"/"+gof_pr_lines['DD'].map(str)+"/"+gof_pr_lines['YYY'].map(str)
gof_pr_lines.head(2)

,Outlet Name,Purchase No.,Pur Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Total Selling,Storeid,Vendor Group,I_U,CTG,CTG PRO,DD,MM,YYY,Purchase Date
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,"1,827.80",3701001,Outside Purchase,10.0,PHARMA,PHARMA,03,04,2023,04/03/2023
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,175.8,3701001,Outside Purchase,1.0,PHARMA,PHARMA,03,04,2023,04/03/2023


In [92]:
gof_pr_lines['Purchase Date'] = gof_pr_lines['Purchase Date'].astype('datetime64[ns]')
gof_pr_lines.head(2)

,Outlet Name,Purchase No.,Pur Date,Invoice No.,Invoice Dt.,VendName,Item Code,Item Name,Batch No.,Expiry Dt.,...,Total Selling,Storeid,Vendor Group,I_U,CTG,CTG PRO,DD,MM,YYY,Purchase Date
0,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,MC961,01-07-2024,...,"1,827.80",3701001,Outside Purchase,10.0,PHARMA,PHARMA,03,04,2023,2023-04-03
1,AP - Currency Nagar,MR317,03-04-2023,R000068,03-04-2023,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,F44Y,01-08-2024,...,175.8,3701001,Outside Purchase,1.0,PHARMA,PHARMA,03,04,2023,2023-04-03


In [93]:
gof_grn=gof_pr_lines.loc[:,['Purchase Date','Storeid', 'Outlet Name','VendName','Item Code', 'Item Name','Vendor Group','CTG PRO','Qty','Item NetAmt']]

In [94]:
gof_grn.rename(columns={'Storeid':"Store",'Outlet Name':"StoreName",'Item Name':"ItemName",'Item NetAmt':"ItemNetAmount"},inplace=True)
gof_grn.head(2)

,Purchase Date,Store,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount
0,2023-04-03,3701001,AP - Currency Nagar,PRUDHVI PHARMACEUTICALS,107009,AUGMENTIN 625 DUO,Outside Purchase,PHARMA,10,"1,374.54"
1,2023-04-03,3701001,AP - Currency Nagar,PRUDHVI PHARMACEUTICALS,139008,BETNOVATE -C SKIN CR,Outside Purchase,PHARMA,3,132.21


In [95]:
#dyn_grn['Qty'] = dyn_grn['Qty'].str.replace(',','').astype(float)
dyn_grn['ItemNetAmount']=dyn_grn['ItemNetAmount'].astype(str)
dyn_grn['ItemNetAmount'] =dyn_grn['ItemNetAmount'].str.replace(',','').astype(float)
dyn_grn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1134746 entries, 0 to 1134745
Data columns (total 10 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   Purchase Date  1134746 non-null  datetime64[ns]
 1   Store          1134746 non-null  object        
 2   StoreName      1134746 non-null  object        
 3   VendName       1134746 non-null  object        
 4   Item Code      1134746 non-null  object        
 5   ItemName       1134746 non-null  object        
 6   Vendor Group   1134746 non-null  object        
 7   CTG PRO        1134746 non-null  object        
 8   Qty            1134746 non-null  float64       
 9   ItemNetAmount  1134746 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 95.2+ MB


In [96]:
gof_grn['Item Code'] = gof_grn['Item Code'].astype(str)
gof_grn['Store'] = gof_grn['Store'].astype(str)
gof_grn['Qty'] = gof_grn['Qty'].str.replace(',','').astype(float)
gof_grn['ItemNetAmount'] = gof_grn['ItemNetAmount'].str.replace(',','').astype(float)
gof_grn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 805086 entries, 0 to 805085
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Purchase Date  805086 non-null  datetime64[ns]
 1   Store          805086 non-null  object        
 2   StoreName      805086 non-null  object        
 3   VendName       805086 non-null  object        
 4   Item Code      805086 non-null  object        
 5   ItemName       805086 non-null  object        
 6   Vendor Group   805086 non-null  object        
 7   CTG PRO        805086 non-null  object        
 8   Qty            805086 non-null  float64       
 9   ItemNetAmount  805086 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(7)
memory usage: 67.6+ MB


In [97]:
pur_report=pd.concat([gof_grn,dyn_grn])
pur_report.reset_index(inplace=True)
pur_report.drop('index',axis=1,inplace=True)
pur_report.columns

Index(['Purchase Date', 'Store', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Qty', 'ItemNetAmount'],
      dtype='object')

In [98]:
purch_rpt=pur_report.pivot_table(index=['Purchase Date', 'Store', 'StoreName', 'VendName', 'Item Code','ItemName', 'Vendor Group', 'CTG PRO'],values=['Qty', 'ItemNetAmount'],aggfunc='sum')
purch_rpt.reset_index(inplace=True)
purch_rpt.head(2)

,Purchase Date,Store,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,ItemNetAmount,Qty
0,2023-04-01,2901003,KA - Sanjay Nagar,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0000036,OKACET COLD TABLET,Generic Distibutor,PHARMA,18.43,1.0
1,2023-04-01,2901003,KA - Sanjay Nagar,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0000057,AZAX 250MG TABLET,Generic Distibutor,PHARMA,39.06,1.0


In [99]:
pur_rpt=purch_rpt.loc[:,['Purchase Date', 'Store', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Qty', 'ItemNetAmount']]

In [100]:
pur_rpt.rename(columns={'Store':"Storeid"},inplace=True)
pur_rpt.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount
0,2023-04-01,2901003,KA - Sanjay Nagar,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0000036,OKACET COLD TABLET,Generic Distibutor,PHARMA,1.0,18.43


In [101]:
sd_details.columns

Index(['S. No.', 'Store Code', 'Store Type', 'HDH Store Code', 'HPD BU2 ID',
       'HPD TNG', 'Get Well', 'SreeJi', 'Vasu agency', 'Venkat Sai',
       'HPD Kerala', 'PO Store Name', 'Store NameAs per Branding ',
       'Outlet Name', 'GOF Store Id', 'Launch Date', 'Email ID', 'Short Name',
       'Build-up Area in Sft', 'Phone Number', 'Ops Exexutive',
       'Cluster Mail Id', 'Executive Phone Number', 'Area Manager',
       'Sub Region', 'State', 'City', 'City 2', 'Gof_status', 'Manager',
       'Cluster', 'Any Desk ID', 'Rental', 'FF Target', 'Address', 'Zip Code',
       'Region', 'HOO', 'Outlet Format', 'Tenure Days', 'Tenure Month',
       'Tenure Years', 'Tenure', 'Launch Month', 'Launch Year', 'Quarter',
       'Financial Year', 'Securty Deposit', 'Rent Start Date', 'Store System',
       'Storeid', 'Store Name As per D 365', 'Retail Channel Id', 'Party ID',
       'Fin And Qtr', 'Location Series', 'City G', 'Vendor Code'],
      dtype='object')

In [102]:
sd_details['Storeid'] = sd_details['Storeid'].astype(str)

In [103]:
pur_rpt=pd.merge(pur_rpt,sd_details[['Storeid','Outlet Name','State','City G','Ops Exexutive','Cluster','Store Type']], on='Storeid', how='left')

In [104]:
pur_rpt.rename(columns={'City G':"City Group"},inplace=True)
pur_rpt.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type
0,2023-04-01,2901003,KA - Sanjay Nagar,HINDUSTAN PHARMA DISTRIBUTORS PRIVATE LIMITED ...,P-0000036,OKACET COLD TABLET,Generic Distibutor,PHARMA,1.0,18.43,KA - Sanjay Nagar,Karnataka,KA,Byregowda MR,CL 4,Retail


In [105]:
purch_reg_rpt=pur_rpt.drop(pur_rpt[pur_rpt['CTG PRO']=='NO DATA'].index)
pr_reg=purch_reg_rpt.drop(purch_reg_rpt[purch_reg_rpt['CTG PRO']=='UN KNOWN'].index)
Purchase_Register=pr_reg.drop(pr_reg[pr_reg['Store Type']=='Warehouse'].index)

In [106]:
Purchase_Register['Storeid'] = Purchase_Register['Storeid'].astype(float)

In [107]:
outside_vendor=Purchase_Register.drop(Purchase_Register[Purchase_Register['Vendor Group']=='Generic Distibutor'].index)
outside_vendor.reset_index(inplace=True)
outside_vendor.drop("index",axis=1,inplace=True)
print(outside_vendor.shape)
outside_vendor.tail(2)

(90462, 16)


,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type
90460,2023-10-25,3601049.0,TG - MCH Park Padmaraonagar,JYOTI MEDICAL HALL,208006,CILACAR 10 MG TAB,Outside Purchase,PHARMA,2.0,291.27,TG - MCH Park Padmaraonagar,Telangana,TG,Ramesh,CL 5,Retail
90461,2023-10-25,3601049.0,TG - MCH Park Padmaraonagar,JYOTI MEDICAL HALL,230026,VYMADA 50 MG TAB,Outside Purchase,PHARMA,1.0,973.09,TG - MCH Park Padmaraonagar,Telangana,TG,Ramesh,CL 5,Retail


In [108]:
outside_vendor.rename(columns={'Item Code':'Pro Code'},inplace=True)
outside_vendor.head(2)

,Purchase Date,Storeid,StoreName,VendName,Pro Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type
0,2023-04-01,3201001.0,KL - Chittoor Road,ABN BUSINESS GROUP,P-0044287,ARIFINE 5MG TABLET,Outside Purchase,PHARMA,3.0,154.60,KL - Chittoor Road,Kerala,KL,Nikhil Antony,CL 3,Retail
1,2023-04-01,3201001.0,KL - Chittoor Road,Local Vendor Kerala,P-0000389,PROHANCE HP VANILLA FLAVOUR JAR POWDER 400GM,Outside Purchase,NON -PHARMA,1.0,865.65,KL - Chittoor Road,Kerala,KL,Nikhil Antony,CL 3,Retail


In [109]:
uniq_master.columns

Index(['Gof Code', 'Gof Description', 'MDM Code OLD', 'MDM Code Revised',
       'Easy Sol Code', 'Easy Sol Desc', '365 Item code', 'D365 Descxription',
       'IU', 'Choose Tax %', 'Sales Blocked', 'Updated Item Name',
       'Special Form', 'Generic Name', 'Strength', 'Form', 'Category 1',
       'Category 2', 'Category 3', 'Category 4', 'Category 5',
       'Category 6 - Brand', 'HSN Codes', 'Long Description', 'GOF CODE',
       'MDM Code', 'Gofcode', 'Easysol', 'D365 Cpde', 'Created Date',
       'Created By', 'CTG', 'KOM', 'CTG 8', 'Updated Pro Name', 'Sch CTG',
       'MDM Rework', 'Medplus Code', 'Act Audit Form', 'Form Name'],
      dtype='object')

In [110]:
uniq_master.rename(columns={'Gof Code':"Pro Code"},inplace =True)

In [111]:
uniq_master['Pro Code'] = uniq_master['Pro Code'].astype(str)

In [112]:
t_day

Timestamp('2023-10-26 00:00:00')

In [113]:
outside_vendor['Purchase Date'].min(),outside_vendor['Purchase Date'].max()

(Timestamp('2023-04-01 00:00:00'), Timestamp('2023-10-25 00:00:00'))

In [114]:
start_date=t_day+timedelta(days=-1)

In [115]:
start_date

Timestamp('2023-10-25 00:00:00')

In [116]:
stop_date = start_date+timedelta(days=-1)

In [117]:
stop_date

Timestamp('2023-10-24 00:00:00')

In [118]:
outside_vendor['start_date']=stop_date

In [119]:
outside_vendor['Days']=outside_vendor['start_date']-outside_vendor['Purchase Date']

In [120]:
outside_purch=outside_vendor.drop(outside_vendor[outside_vendor['Days']>'30 Days'].index)

In [121]:
outside_purch.reset_index(drop=True,inplace=True)
outside_purch=outside_purch.drop(outside_purch[outside_purch['Days']<'0 Days'].index)
outside_purch.reset_index(drop=True,inplace=True)

In [122]:
outside_purch.reset_index(inplace=True)
outside_purch.drop("index",axis=1,inplace=True)

In [123]:
print(outside_purch.shape)
outside_purch.tail(2)

(4857, 18)


,Purchase Date,Storeid,StoreName,VendName,Pro Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days
4855,2023-10-24,3601058.0,TG - Ayodhya Nagar Colony Mehdipatnam,PRIYANKA PHARMA,920816,RENODAPT S 360 TABLET,Outside Purchase,PHARMA,1.0,852.00,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,TG,Renatla Surender,CL 5,Retail,2023-10-24,0 days
4856,2023-10-24,3601058.0,TG - Ayodhya Nagar Colony Mehdipatnam,PRIYANKA PHARMA,928004,SCALPE PRO 100ML SHAMPOO,Outside Purchase,PHARMA,1.0,165.74,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,TG,Renatla Surender,CL 5,Retail,2023-10-24,0 days


In [124]:
outside_purch=pd.merge(outside_purch,uniq_master[['Pro Code','MDM Code']],on='Pro Code',how='left')
outside_purch.head(2)

,Purchase Date,Storeid,StoreName,VendName,Pro Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days,MDM Code
0,2023-09-24,3201021.0,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,KL - Sub Jail Road Aluva,Kerala,KL,Boney Louies,CL 6,Retail,2023-10-24,30 days,NaN
1,2023-09-24,3201021.0,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0024340,WAKCERIN GM TABLET 10S,Outside Purchase,PHARMA,1.0,140.91,KL - Sub Jail Road Aluva,Kerala,KL,Boney Louies,CL 6,Retail,2023-10-24,30 days,NaN


In [125]:
outside_purch.rename(columns={'Pro Code':"Item Code"},inplace=True)
outside_purch=pd.merge(outside_purch,dyn_pro[['Item Code','One MDM Code']],on='Item Code',how='left')

In [126]:
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days,MDM Code,One MDM Code
0,2023-09-24,3201021.0,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,KL - Sub Jail Road Aluva,Kerala,KL,Boney Louies,CL 6,Retail,2023-10-24,30 days,NaN,3-A-CA55-HW-CZ45-AR35-X-00001


In [127]:
outside_purch.rename(columns={'Pro Code':"Item Code"},inplace=True)
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days,MDM Code,One MDM Code
0,2023-09-24,3201021.0,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,KL - Sub Jail Road Aluva,Kerala,KL,Boney Louies,CL 6,Retail,2023-10-24,30 days,NaN,3-A-CA55-HW-CZ45-AR35-X-00001


In [128]:
gof_pro['Item Code']=gof_pro['Item Code'].astype(int)

In [129]:
gof_pro['Item Code']=gof_pro['Item Code'].astype(str)

In [130]:
gof_pro.head()

,Item Code,Item Name,Manufacturer Name,CTG,Therapeutic,BOM,Medical Item Type,KOM,Generic Category,Generic Name,Main Supplier Name,I_U,Strength,Choose Tax,Basic Cost,Selling Price,MRP,Purchase Price
1,1,PANTOSEC 40 MG TABLET 10S,CIPLA GENERIC (AAGAM ),PHARMA,NaN,NaN,1-Scheduled,FMCG,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,10.0,.,GST 12%,0.0,0.00,0.00,0.0
2,3,NORLUT N 5MG TAB 10S,CIPLA GENERICS(M P),PHARMA,NaN,NaN,1-Scheduled,GENERIC,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,10.0,.,GST 12%,0.0,0.00,0.00,0.0
3,5,CIPLADINE OINTMENT 10GM,CIPLA GENERIC (AAGAM ),PHARMA,NaN,NaN,1-Scheduled,GENERIC,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,1.0,.,GST 12%,0.0,0.00,0.00,0.0
4,7,COF Q TABLET 10S,CIPLA GENERIC (AAGAM ),PHARMA,NaN,NaN,1-Scheduled,GENERIC,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,10.0,.,GST 12%,0.0,0.00,0.00,0.0
5,8,SUHAGRA 50 TABLET 4S,CIPLA GENERICS(M P),PHARMA,NaN,NaN,1-Scheduled,GENERIC,ALLOPATHIC,NaN,HINDUSTAN DRUG HOUSE,4.0,.,GST 12%,0.0,0.00,0.00,0.0


In [131]:
outside_purch['Item Code'].dtype

dtype('O')

In [132]:
outside_purch=pd.merge(outside_purch,dyn_pro[['Item Code','Unit']],on='Item Code',how='left')

In [133]:
outside_purch['Unit'] = outside_purch['Unit'].str.replace("'s",'').astype(str)
outside_purch['Unit'] = outside_purch['Unit'].str.replace("'S",'').astype(str)
outside_purch['Unit'] = outside_purch['Unit'].str.replace("ea","1").astype(float)

In [134]:
outside_purch=pd.merge(outside_purch,gof_pro[['Item Code','I_U']],on='Item Code',how='left')

In [135]:
outside_purch["Unit"].fillna(outside_purch["I_U"], inplace = True)
outside_purch["One MDM Code"].fillna(outside_purch["MDM Code"], inplace = True)

In [136]:
outside_purch.drop(columns=["I_U"],inplace=True)
outside_purch.drop(columns=["MDM Code"],inplace=True)

###### Stock In Transfer

In [137]:
gof_transit=pd.read_csv('D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Aster Pharmacy Raw data\Stock in Transit\\40000256_Stock_In_T.csv'
                       ,skiprows=3,low_memory=False)
gof_transit.drop(gof_transit.index[0],inplace=True)

dyn_transit=pd.read_csv('D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Aster Pharmacy Raw data\Stock in Transit\Internal Branch Transfer.csv'
                        ,low_memory=False)

In [138]:
gof_transit.head(1)

,CATEGORY,THERAPEUTIC CLASSIFI,BRANCH OF MEDICINE,MEDICAL ITEM TYPE,KIND OF MEDICINE,GENERIC CATEGORY,GENERIC NAME,DISEASE TYPE,Source Outlet Name,Destination Outlet Name,...,Item Code,Item Name,Supplier DC No,TO Date,Item Qty,Item Qty(Unit),Item Amt,Item Amt(WO Tax),Tax Amt,Status
1,AYURVEDIC,NaN,NaN,3-Others,CONTAINER,NORMAL,NaN,FMCG,TG - Srinagar Colony,TG - Yousufguda Check Post,...,537407.0,KAYAM TABLET 30S,432.0,2023-09-24,2.00,2.00,179.02,179.02,0.0,Transferred


In [139]:
dyn_transit.head(1)

,FromStore,FromStoreName,ToNo,TransDate,ToStore,ToStoreName,Itemid,ItemName,ToQty,TOValue,TransferTypeName,StatusName1,Textbox69,Textbox70
0,2901001,Dasarahalli,TO-020812,01-01-2023,2901059,Kodigehalli Main Road,P-0005179,MIXTARD 50 HM PENFILL 100IU 3ML INJ,4.0,854.88,StockTransfer,Received,5281839.0,1.007847e+08


###### Stock in Transit Gofrugal Cleaning

In [140]:
st_store_map.columns

Index(['Store ID', 'Destination Outlet Name', 'Store Type'], dtype='object')

In [141]:
gof_transit.columns

Index(['CATEGORY', 'THERAPEUTIC CLASSIFI', 'BRANCH OF MEDICINE',
       'MEDICAL ITEM TYPE', 'KIND OF MEDICINE', 'GENERIC CATEGORY',
       'GENERIC NAME', 'DISEASE TYPE', 'Source Outlet Name',
       'Destination Outlet Name', 'Company_Name', 'Division_Name',
       'Location_Name', 'Combination ID', 'Item Code', 'Item Name',
       'Supplier DC No', 'TO Date', 'Item Qty', 'Item Qty(Unit)', 'Item Amt',
       'Item Amt(WO Tax)', 'Tax Amt', 'Status'],
      dtype='object')

In [142]:
gof_transit=pd.merge(gof_transit,st_store_map[['Destination Outlet Name','Store ID']],on='Destination Outlet Name',how='left')
gof_transit.rename(columns={'Store ID':"Destination Storeid"},inplace=True)

In [143]:
gof_transit['Destination Storeid'].fillna(0, inplace = True)
gof_transit['Destination Storeid']=gof_transit['Destination Storeid'].astype(int)

In [144]:
gof_transit.dtypes

CATEGORY                    object
THERAPEUTIC CLASSIFI        object
BRANCH OF MEDICINE          object
MEDICAL ITEM TYPE           object
KIND OF MEDICINE            object
GENERIC CATEGORY            object
GENERIC NAME               float64
DISEASE TYPE                object
Source Outlet Name          object
Destination Outlet Name     object
Company_Name                object
Division_Name               object
Location_Name               object
Combination ID             float64
Item Code                  float64
Item Name                   object
Supplier DC No             float64
TO Date                     object
Item Qty                    object
Item Qty(Unit)              object
Item Amt                    object
Item Amt(WO Tax)            object
Tax Amt                    float64
Status                      object
Destination Storeid          int32
dtype: object

In [145]:
gof_transit.head(1)

,CATEGORY,THERAPEUTIC CLASSIFI,BRANCH OF MEDICINE,MEDICAL ITEM TYPE,KIND OF MEDICINE,GENERIC CATEGORY,GENERIC NAME,DISEASE TYPE,Source Outlet Name,Destination Outlet Name,...,Item Name,Supplier DC No,TO Date,Item Qty,Item Qty(Unit),Item Amt,Item Amt(WO Tax),Tax Amt,Status,Destination Storeid
0,AYURVEDIC,NaN,NaN,3-Others,CONTAINER,NORMAL,NaN,FMCG,TG - Srinagar Colony,TG - Yousufguda Check Post,...,KAYAM TABLET 30S,432.0,2023-09-24,2.00,2.00,179.02,179.02,0.0,Transferred,3601047


In [146]:
st_store_map.rename(columns={'Destination Outlet Name':'Source Outlet Name'},inplace=True)

In [147]:
st_store_map.columns

Index(['Store ID', 'Source Outlet Name', 'Store Type'], dtype='object')

In [148]:
gof_transit=pd.merge(gof_transit,st_store_map[['Source Outlet Name','Store ID']],on='Source Outlet Name',how='left')
gof_transit.rename(columns={'Store ID':"Source Storeid"},inplace=True)

In [149]:
gof_transit['Source Storeid'].fillna(0, inplace = True)
gof_transit['Source Storeid']=gof_transit['Source Storeid'].astype(int)

In [150]:
gof_transit.head(1)

,CATEGORY,THERAPEUTIC CLASSIFI,BRANCH OF MEDICINE,MEDICAL ITEM TYPE,KIND OF MEDICINE,GENERIC CATEGORY,GENERIC NAME,DISEASE TYPE,Source Outlet Name,Destination Outlet Name,...,Supplier DC No,TO Date,Item Qty,Item Qty(Unit),Item Amt,Item Amt(WO Tax),Tax Amt,Status,Destination Storeid,Source Storeid
0,AYURVEDIC,NaN,NaN,3-Others,CONTAINER,NORMAL,NaN,FMCG,TG - Srinagar Colony,TG - Yousufguda Check Post,...,432.0,2023-09-24,2.00,2.00,179.02,179.02,0.0,Transferred,3601047,3601011


In [151]:
gof_transit['System']="Gofrugal"

In [152]:
gof_transit.head(2)

,CATEGORY,THERAPEUTIC CLASSIFI,BRANCH OF MEDICINE,MEDICAL ITEM TYPE,KIND OF MEDICINE,GENERIC CATEGORY,GENERIC NAME,DISEASE TYPE,Source Outlet Name,Destination Outlet Name,...,TO Date,Item Qty,Item Qty(Unit),Item Amt,Item Amt(WO Tax),Tax Amt,Status,Destination Storeid,Source Storeid,System
0,AYURVEDIC,NaN,NaN,3-Others,CONTAINER,NORMAL,NaN,FMCG,TG - Srinagar Colony,TG - Yousufguda Check Post,...,2023-09-24,2.00,2.00,179.02,179.02,0.0,Transferred,3601047,3601011,Gofrugal
1,AYURVEDIC,NaN,NaN,3-Others,CONTAINER,AYURVEDIC,NaN,ACUTE,TG - Sanath Nagar,TG - Warehouse,...,2023-10-13,1.00,1.00,123.59,123.59,0.0,Stock in Transit,3690002,3601046,Gofrugal


In [153]:
gof_check=gof_transit.loc[:,['Source Outlet Name','Destination Outlet Name','Destination Storeid','Source Storeid']]

In [154]:
gof_source_unq=gof_check.drop_duplicates(subset = 'Source Outlet Name')
gof_source_check=gof_source_unq.drop(gof_source_unq[gof_source_unq['Source Storeid']!=0].index)

In [155]:
gof_destination_unq=gof_check.drop_duplicates(subset = 'Destination Outlet Name')
gof_destination_check=gof_source_unq.drop(gof_source_unq[gof_source_unq['Destination Storeid']!=0].index)

In [156]:
gof_source_check.head(2)

,Source Outlet Name,Destination Outlet Name,Destination Storeid,Source Storeid


In [157]:
gof_destination_check.head(2)

,Source Outlet Name,Destination Outlet Name,Destination Storeid,Source Storeid


In [158]:
date=time.strftime("%d-%m-%Y")

In [159]:
missingfile=missed_rd+"Missing Store in stock in transit "+date+".xlsx"

In [160]:
missingfile

'D:\\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\\Aster Pharmacy Raw data\\Phyton RD Drive\\Missing RD\\Missing Store in stock in transit 26-10-2023.xlsx'

In [161]:
with pd.ExcelWriter(missingfile,engine="openpyxl") as writer:
    gof_source_check.to_excel(writer,sheet_name='Summary',index=False)
    gof_destination_check.to_excel(writer,sheet_name='Summary',startcol=7,startrow=0,index=False)    

In [162]:
trasitcheck=input()

In [163]:
gof_transit.columns,dyn_transit.columns

(Index(['CATEGORY', 'THERAPEUTIC CLASSIFI', 'BRANCH OF MEDICINE',
        'MEDICAL ITEM TYPE', 'KIND OF MEDICINE', 'GENERIC CATEGORY',
        'GENERIC NAME', 'DISEASE TYPE', 'Source Outlet Name',
        'Destination Outlet Name', 'Company_Name', 'Division_Name',
        'Location_Name', 'Combination ID', 'Item Code', 'Item Name',
        'Supplier DC No', 'TO Date', 'Item Qty', 'Item Qty(Unit)', 'Item Amt',
        'Item Amt(WO Tax)', 'Tax Amt', 'Status', 'Destination Storeid',
        'Source Storeid', 'System'],
       dtype='object'),
 Index(['FromStore', 'FromStoreName', 'ToNo', 'TransDate', 'ToStore',
        'ToStoreName', 'Itemid', 'ItemName', 'ToQty', 'TOValue',
        'TransferTypeName', 'StatusName1', 'Textbox69', 'Textbox70'],
       dtype='object'))

In [164]:
gof_stk_in_transit=gof_transit.loc[:,['Source Storeid','Supplier DC No','TO Date','Destination Storeid','Item Code'
                                     , 'Item Name','Item Qty','Item Amt(WO Tax)','Status','System']]

In [165]:
gof_stk_in_transit.head(1)

,Source Storeid,Supplier DC No,TO Date,Destination Storeid,Item Code,Item Name,Item Qty,Item Amt(WO Tax),Status,System
0,3601011,432.0,2023-09-24,3601047,537407.0,KAYAM TABLET 30S,2.00,179.02,Transferred,Gofrugal


###### Stock in Transit Dynamics Cleaning

In [166]:
dyn_stk_in_transit=dyn_transit.loc[:,['FromStore','ToNo','TransDate','ToStore','Itemid','ItemName','ToQty','TOValue'
                                     ,'StatusName1']]

In [167]:
dyn_stk_in_transit['System']="Dynamics"

In [168]:
dyn_stk_in_transit.head(2)

,FromStore,ToNo,TransDate,ToStore,Itemid,ItemName,ToQty,TOValue,StatusName1,System
0,2901001,TO-020812,01-01-2023,2901059,P-0005179,MIXTARD 50 HM PENFILL 100IU 3ML INJ,4.0,854.88,Received,Dynamics
1,2901001,TO-020814,01-01-2023,2901003,P-0005613,ACCUSURE HS INFRARED THERMOMETER,2.0,1300.00,Received,Dynamics


In [169]:
gof_stk_in_transit.rename(columns={'Source Storeid':"FromStore", 'Supplier DC No':"ToNo", 'TO Date':"TransDate", 'Destination Storeid':"ToStore",'Item Code':"Itemid", 'Item Qty':"ToQty", 'Item Amt(WO Tax)':"TOValue"},inplace=True)

In [170]:
dyn_stk_in_transit.rename(columns={'StatusName1':'Status','ItemName':'Item Name'},inplace=True)

In [171]:
gof_stk_in_transit.columns,dyn_stk_in_transit.columns

(Index(['FromStore', 'ToNo', 'TransDate', 'ToStore', 'Itemid', 'Item Name',
        'ToQty', 'TOValue', 'Status', 'System'],
       dtype='object'),
 Index(['FromStore', 'ToNo', 'TransDate', 'ToStore', 'Itemid', 'Item Name',
        'ToQty', 'TOValue', 'Status', 'System'],
       dtype='object'))

In [172]:
gof_stk_in_transit['ToQty']=gof_stk_in_transit['ToQty'].str.replace(",","").astype(float)
gof_stk_in_transit['TOValue']=gof_stk_in_transit['TOValue'].str.replace(",","").astype(float)

In [173]:
gof_stk_in_transit.dtypes

FromStore      int32
ToNo         float64
TransDate     object
ToStore        int32
Itemid       float64
Item Name     object
ToQty        float64
TOValue      float64
Status        object
System        object
dtype: object

In [174]:
dyn_stk_in_transit[['DD','MM','YYY']]=dyn_stk_in_transit['TransDate'].str.split("-",expand=True)
dyn_stk_in_transit.head(2)

,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System,DD,MM,YYY
0,2901001,TO-020812,01-01-2023,2901059,P-0005179,MIXTARD 50 HM PENFILL 100IU 3ML INJ,4.0,854.88,Received,Dynamics,01,01,2023
1,2901001,TO-020814,01-01-2023,2901003,P-0005613,ACCUSURE HS INFRARED THERMOMETER,2.0,1300.00,Received,Dynamics,01,01,2023


In [175]:
dyn_stk_in_transit['TransDate'] = dyn_stk_in_transit['YYY'].map(str)+"-"+dyn_stk_in_transit['MM'].map(str)+"-"+dyn_stk_in_transit['DD'].map(str)

In [176]:
dyn_stk_in_transit.drop(columns=["DD","MM","YYY"],inplace=True)

In [177]:
dyn_stk_in_transit.head()

,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System
0,2901001,TO-020812,2023-01-01,2901059,P-0005179,MIXTARD 50 HM PENFILL 100IU 3ML INJ,4.0,854.88,Received,Dynamics
1,2901001,TO-020814,2023-01-01,2901003,P-0005613,ACCUSURE HS INFRARED THERMOMETER,2.0,1300.00,Received,Dynamics
2,2901001,TO-020815,2023-01-01,2901004,P-0000812,DORZOX T EYE DROPS 5ML,1.0,278.29,Received,Dynamics
3,2901001,TO-020815,2023-01-01,2901004,P-0000811,DORZOX EYE DROPS 5ML,1.0,327.88,Received,Dynamics
4,2901001,TO-020815,2023-01-01,2901004,P-0004818,ISTAMET 50MG/500MG TABLET,30.0,552.90,Received,Dynamics


In [178]:
gof_stk_in_transit.head()

,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System
0,3601011,432.0,2023-09-24,3601047,537407.0,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal
1,3601046,353.0,2023-10-13,3690002,110005.0,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal
2,3601051,102.0,2023-05-20,3601048,110015.0,LIV - 52 SYRUP 200ML,2.0,203.28,Transferred,Gofrugal
3,3601049,255.0,2023-09-26,3690002,110021.0,SEPTILIN TABLETS,1.0,119.41,Stock in Transit,Gofrugal
4,3601049,257.0,2023-09-27,3690002,110029.0,HIMCOCID SAUNF FLA (SF) 200ML,1.0,96.56,Stock in Transit,Gofrugal


In [179]:
gof_stk_in_transit['FromStore']=gof_stk_in_transit['FromStore'].astype(str)
gof_stk_in_transit['ToStore']=gof_stk_in_transit['ToStore'].astype(str)
gof_stk_in_transit['Itemid']=gof_stk_in_transit['Itemid'].astype(int).astype(str)
gof_stk_in_transit.dtypes

FromStore     object
ToNo         float64
TransDate     object
ToStore       object
Itemid        object
Item Name     object
ToQty        float64
TOValue      float64
Status        object
System        object
dtype: object

In [180]:
dyn_stk_in_transit['FromStore']=dyn_stk_in_transit['FromStore'].astype(str)
dyn_stk_in_transit.dtypes

FromStore     object
ToNo          object
TransDate     object
ToStore        int64
Itemid        object
Item Name     object
ToQty        float64
TOValue      float64
Status        object
System        object
dtype: object

###### Stock in transit Consolidate

In [181]:
stock_in_transit=pd.concat([gof_stk_in_transit,dyn_stk_in_transit])
stock_in_transit.reset_index(inplace=True)
stock_in_transit.drop('index',axis=1,inplace=True)
stock_in_transit.head()

,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal
1,3601046,353.0,2023-10-13,3690002,110005,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal
2,3601051,102.0,2023-05-20,3601048,110015,LIV - 52 SYRUP 200ML,2.0,203.28,Transferred,Gofrugal
3,3601049,255.0,2023-09-26,3690002,110021,SEPTILIN TABLETS,1.0,119.41,Stock in Transit,Gofrugal
4,3601049,257.0,2023-09-27,3690002,110029,HIMCOCID SAUNF FLA (SF) 200ML,1.0,96.56,Stock in Transit,Gofrugal


In [182]:
print(gof_stk_in_transit.shape)
print(dyn_stk_in_transit.shape)
print(stock_in_transit.shape)
stock_in_transit.head()

(104114, 10)
(376199, 10)
(480313, 10)


,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal
1,3601046,353.0,2023-10-13,3690002,110005,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal
2,3601051,102.0,2023-05-20,3601048,110015,LIV - 52 SYRUP 200ML,2.0,203.28,Transferred,Gofrugal
3,3601049,255.0,2023-09-26,3690002,110021,SEPTILIN TABLETS,1.0,119.41,Stock in Transit,Gofrugal
4,3601049,257.0,2023-09-27,3690002,110029,HIMCOCID SAUNF FLA (SF) 200ML,1.0,96.56,Stock in Transit,Gofrugal


In [183]:
stock_in_transit['TransDate']=stock_in_transit['TransDate'].astype('datetime64')
stock_in_transit.dtypes

FromStore            object
ToNo                 object
TransDate    datetime64[ns]
ToStore              object
Itemid               object
Item Name            object
ToQty               float64
TOValue             float64
Status               object
System               object
dtype: object

In [184]:
stock_in_transit.columns,sd_details.columns

(Index(['FromStore', 'ToNo', 'TransDate', 'ToStore', 'Itemid', 'Item Name',
        'ToQty', 'TOValue', 'Status', 'System'],
       dtype='object'),
 Index(['S. No.', 'Store Code', 'Store Type', 'HDH Store Code', 'HPD BU2 ID',
        'HPD TNG', 'Get Well', 'SreeJi', 'Vasu agency', 'Venkat Sai',
        'HPD Kerala', 'PO Store Name', 'Store NameAs per Branding ',
        'Outlet Name', 'GOF Store Id', 'Launch Date', 'Email ID', 'Short Name',
        'Build-up Area in Sft', 'Phone Number', 'Ops Exexutive',
        'Cluster Mail Id', 'Executive Phone Number', 'Area Manager',
        'Sub Region', 'State', 'City', 'City 2', 'Gof_status', 'Manager',
        'Cluster', 'Any Desk ID', 'Rental', 'FF Target', 'Address', 'Zip Code',
        'Region', 'HOO', 'Outlet Format', 'Tenure Days', 'Tenure Month',
        'Tenure Years', 'Tenure', 'Launch Month', 'Launch Year', 'Quarter',
        'Financial Year', 'Securty Deposit', 'Rent Start Date', 'Store System',
        'Storeid', 'Store Name As per

In [185]:
stock_in_transit=pd.merge(stock_in_transit,sd_details[['Storeid','Outlet Name','Ops Exexutive', 'City 2'
                                                       , 'Manager']],left_on='FromStore',right_on='Storeid',how='left')

In [186]:
print(stock_in_transit.shape)
stock_in_transit.rename(columns={'Outlet Name':'From Outlet Name','City 2':"From City",'Ops Exexutive':'From Executive'},inplace=True)
stock_in_transit.drop(columns=['Storeid'],inplace=True)
print(stock_in_transit.shape)
stock_in_transit.head(2)

(480313, 15)
(480313, 14)


,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System,From Outlet Name,From Executive,From City,Manager
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal,TG - Srinagar Colony,Niwin,TNG,Pallerla Chinnap Reddy
1,3601046,353.0,2023-10-13,3690002,110005,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal,TG - Sanath Nagar,Ephraim,TNG,Pallerla Chinnap Reddy


In [187]:
stock_in_transit['Status Remarks']=stock_in_transit['Status']

In [188]:
stock_in_transit['Status Remarks']=stock_in_transit['Status Remarks'].str.replace('Stock in Transit','Shipped')

In [189]:
stock_in_transit.head(2)

,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System,From Outlet Name,From Executive,From City,Manager,Status Remarks
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal,TG - Srinagar Colony,Niwin,TNG,Pallerla Chinnap Reddy,Transferred
1,3601046,353.0,2023-10-13,3690002,110005,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal,TG - Sanath Nagar,Ephraim,TNG,Pallerla Chinnap Reddy,Shipped


In [190]:
stock_in_transit=pd.merge(stock_in_transit,sd_details[['Storeid','Outlet Name','City 2','Ops Exexutive']],left_on='ToStore',right_on='Storeid',how='left')

In [191]:
print(stock_in_transit.shape)
stock_in_transit.rename(columns={'Outlet Name':'To Outlet Name','City 2':"To City",'Ops Exexutive':'TO Executive'},inplace=True)
stock_in_transit.drop(columns=['Storeid'],inplace=True)
print(stock_in_transit.shape)
stock_in_transit.head(2)

(480313, 19)
(480313, 18)


,FromStore,ToNo,TransDate,ToStore,Itemid,Item Name,ToQty,TOValue,Status,System,From Outlet Name,From Executive,From City,Manager,Status Remarks,To Outlet Name,To City,TO Executive
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal,TG - Srinagar Colony,Niwin,TNG,Pallerla Chinnap Reddy,Transferred,TG - Yousufguda Check Post,TNG,Niwin
1,3601046,353.0,2023-10-13,3690002,110005,GASEX TABLETS,1.0,123.59,Stock in Transit,Gofrugal,TG - Sanath Nagar,Ephraim,TNG,Pallerla Chinnap Reddy,Shipped,TG - Warehouse,TNG Ware House,warehouse


###### Op Purchase & IBT Transfer Data

where It will Create duplicates Lines

In [192]:
stock_in_transit.shape

(480313, 18)

In [193]:
stock_in_transit['TO Executive'].value_counts()[:1]

warehouse    41877
Name: TO Executive, dtype: int64

In [194]:
ibt=stock_in_transit.drop(stock_in_transit.loc[stock_in_transit['TO Executive']=='warehouse'].index)
print(ibt.shape)

(438436, 18)


In [195]:
ibt.reset_index(inplace=True)
ibt.drop('index',axis=1,inplace=True)

In [196]:
print(ibt.shape)
ibt.columns

(438436, 18)


Index(['FromStore', 'ToNo', 'TransDate', 'ToStore', 'Itemid', 'Item Name',
       'ToQty', 'TOValue', 'Status', 'System', 'From Outlet Name',
       'From Executive', 'From City', 'Manager', 'Status Remarks',
       'To Outlet Name', 'To City', 'TO Executive'],
      dtype='object')

In [197]:
ibt['FromStore']=ibt['FromStore'].astype(str)
ibt['FromStore'].dtypes

dtype('O')

In [198]:
outside_purch['Storeid']=outside_purch['Storeid'].astype(int).astype(str)

In [199]:
op_purch=outside_purch.copy()

In [200]:
op_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days,One MDM Code,Unit
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,KL - Sub Jail Road Aluva,Kerala,KL,Boney Louies,CL 6,Retail,2023-10-24,30 days,3-A-CA55-HW-CZ45-AR35-X-00001,10.0


In [201]:
ibt.rename(columns={'FromStore':"Storeid",'Itemid':"Item Code"},inplace=True)

In [202]:
ibt.head(1)

,Storeid,ToNo,TransDate,ToStore,Item Code,Item Name,ToQty,TOValue,Status,System,From Outlet Name,From Executive,From City,Manager,Status Remarks,To Outlet Name,To City,TO Executive
0,3601011,432.0,2023-09-24,3601047,537407,KAYAM TABLET 30S,2.0,179.02,Transferred,Gofrugal,TG - Srinagar Colony,Niwin,TNG,Pallerla Chinnap Reddy,Transferred,TG - Yousufguda Check Post,TNG,Niwin


In [203]:
ibt_details=pd.merge(ibt,op_purch,on=['Storeid','Item Code'])

In [204]:
ibt_details.columns

Index(['Storeid', 'ToNo', 'TransDate', 'ToStore', 'Item Code', 'Item Name',
       'ToQty', 'TOValue', 'Status', 'System', 'From Outlet Name',
       'From Executive', 'From City', 'Manager', 'Status Remarks',
       'To Outlet Name', 'To City', 'TO Executive', 'Purchase Date',
       'StoreName', 'VendName', 'ItemName', 'Vendor Group', 'CTG PRO', 'Qty',
       'ItemNetAmount', 'Outlet Name', 'State', 'City Group', 'Ops Exexutive',
       'Cluster', 'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit'],
      dtype='object')

In [205]:
ibt_details.head(1)

,Storeid,ToNo,TransDate,ToStore,Item Code,Item Name,ToQty,TOValue,Status,System,...,Outlet Name,State,City Group,Ops Exexutive,Cluster,Store Type,start_date,Days,One MDM Code,Unit
0,3601006,848.0,2023-10-10,3601058,110012,LIV-52 DS TABS,1.0,139.37,Transferred,Gofrugal,...,TG - Anand Nagar Khairatabad,Telangana,TG,Niwin,CL 6,Retail,2023-10-24,11 days,3-K-BM4-EO4-CK76-XX09-00036,1.0


In [206]:
ibt_details.columns

Index(['Storeid', 'ToNo', 'TransDate', 'ToStore', 'Item Code', 'Item Name',
       'ToQty', 'TOValue', 'Status', 'System', 'From Outlet Name',
       'From Executive', 'From City', 'Manager', 'Status Remarks',
       'To Outlet Name', 'To City', 'TO Executive', 'Purchase Date',
       'StoreName', 'VendName', 'ItemName', 'Vendor Group', 'CTG PRO', 'Qty',
       'ItemNetAmount', 'Outlet Name', 'State', 'City Group', 'Ops Exexutive',
       'Cluster', 'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit'],
      dtype='object')

In [207]:
for ind,row in ibt_details.iterrows():
    if row['TransDate']<row['Purchase Date']:
        ibt_details.loc[ind,'Total Trasanferd qty']=0
    else:
        ibt_details.loc[ind,'Total Trasanferd qty']=row['ToQty']

In [208]:
for ind,row in ibt_details.iterrows():
    if row['Total Trasanferd qty']<=0:
        ibt_details.loc[ind,'Istrasffered']="No"
    else:
        ibt_details.loc[ind,'Istrasffered']="Yes"

In [209]:
ibt_details.columns

Index(['Storeid', 'ToNo', 'TransDate', 'ToStore', 'Item Code', 'Item Name',
       'ToQty', 'TOValue', 'Status', 'System', 'From Outlet Name',
       'From Executive', 'From City', 'Manager', 'Status Remarks',
       'To Outlet Name', 'To City', 'TO Executive', 'Purchase Date',
       'StoreName', 'VendName', 'ItemName', 'Vendor Group', 'CTG PRO', 'Qty',
       'ItemNetAmount', 'Outlet Name', 'State', 'City Group', 'Ops Exexutive',
       'Cluster', 'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit',
       'Total Trasanferd qty', 'Istrasffered'],
      dtype='object')

In [210]:
ibt_to_sc=ibt_details.loc[ibt_details['Istrasffered']=="Yes"].pivot_table(index=['Storeid','TransDate','ToStore','Item Code','Purchase Date'],values='Total Trasanferd qty',aggfunc='sum')

In [211]:
ibt_to_sc.reset_index(inplace=True)
print(ibt_to_sc.shape)
ibt_to_sc.head()

(134, 6)


,Storeid,TransDate,ToStore,Item Code,Purchase Date,Total Trasanferd qty
0,2901001,2023-10-05,2901076,P-0014904,2023-10-05,1.0
1,2907004,2023-10-06,2907003,P-0032947,2023-10-06,60.0
2,2907004,2023-10-06,2907003,P-0040862,2023-10-06,3.0
3,2907004,2023-10-06,2907003,P-0087159,2023-10-06,30.0
4,2907004,2023-10-06,2907003,P-0087737,2023-10-06,30.0


In [212]:
ibt_to_sc['ToStore']=ibt_to_sc['ToStore'].astype(str)

In [213]:
ibt_to_sc['Tra Store id']=ibt_to_sc['ToStore']+"|"

In [214]:
ibt_to_sc['T-date']=ibt_to_sc['TransDate'].astype(str)+","

In [215]:
ibt_look=ibt_to_sc.pivot_table(index=['Storeid','Item Code','Purchase Date'],values=['Tra Store id','T-date'],aggfunc='sum')

In [216]:
ibt_look.reset_index(inplace=True)

In [217]:
print(ibt_look.shape)
ibt_look.head()

(128, 5)


,Storeid,Item Code,Purchase Date,T-date,Tra Store id
0,2901001,P-0014904,2023-10-05,"2023-10-05,",2901076|
1,2907004,P-0004250,2023-10-06,"2023-10-17,",2907002|
2,2907004,P-0016030,2023-10-06,"2023-10-17,",2907002|
3,2907004,P-0032947,2023-10-06,"2023-10-06,",2907003|
4,2907004,P-0040862,2023-10-06,"2023-10-06,",2907003|


In [218]:
ibt_trans_qty=ibt_to_sc.pivot_table(index=['Storeid','Item Code','Purchase Date'],values='Total Trasanferd qty',aggfunc='sum')

In [219]:
ibt_trans_qty.reset_index(inplace=True)
print(ibt_trans_qty.shape)
ibt_trans_qty.head()

(128, 4)


,Storeid,Item Code,Purchase Date,Total Trasanferd qty
0,2901001,P-0014904,2023-10-05,1.0
1,2907004,P-0004250,2023-10-06,30.0
2,2907004,P-0016030,2023-10-06,30.0
3,2907004,P-0032947,2023-10-06,60.0
4,2907004,P-0040862,2023-10-06,3.0


In [220]:
ibt_look=pd.merge(ibt_look,ibt_trans_qty)

In [221]:
print(ibt_look.shape)

(128, 6)


In [222]:
ibt_look.head()

,Storeid,Item Code,Purchase Date,T-date,Tra Store id,Total Trasanferd qty
0,2901001,P-0014904,2023-10-05,"2023-10-05,",2901076|,1.0
1,2907004,P-0004250,2023-10-06,"2023-10-17,",2907002|,30.0
2,2907004,P-0016030,2023-10-06,"2023-10-17,",2907002|,30.0
3,2907004,P-0032947,2023-10-06,"2023-10-06,",2907003|,60.0
4,2907004,P-0040862,2023-10-06,"2023-10-06,",2907003|,3.0


In [223]:
outside_purch.columns,ibt_look.columns

(Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
        'ItemName', 'Vendor Group', 'CTG PRO', 'Qty', 'ItemNetAmount',
        'Outlet Name', 'State', 'City Group', 'Ops Exexutive', 'Cluster',
        'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit'],
       dtype='object'),
 Index(['Storeid', 'Item Code', 'Purchase Date', 'T-date', 'Tra Store id',
        'Total Trasanferd qty'],
       dtype='object'))

In [224]:
outside_purch=pd.merge(outside_purch,ibt_look[['Purchase Date', 'Storeid','Item Code','T-date', 'Tra Store id','Total Trasanferd qty']],on=['Storeid', 'Item Code', 'Purchase Date'],how='left')

In [225]:
print(outside_purch.shape)
outside_purch.head(1)

(4857, 23)


,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,...,Ops Exexutive,Cluster,Store Type,start_date,Days,One MDM Code,Unit,T-date,Tra Store id,Total Trasanferd qty
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,Boney Louies,CL 6,Retail,2023-10-24,30 days,3-A-CA55-HW-CZ45-AR35-X-00001,10.0,NaN,NaN,NaN


In [226]:
outside_purch["Tra Store id"]=outside_purch['Tra Store id'].str.slice(0,-1,1)

In [227]:
outside_purch["Total Trasanferd qty"].fillna(0,inplace=True)

In [228]:
outside_purch["Overall"]=1

In [229]:
for ind,row in outside_purch.iterrows():
    if row["Total Trasanferd qty"]>0:
        outside_purch.loc[ind,"Xfer(D) & Recv(D)in Tostore" ]=1
    else:
        outside_purch.loc[ind,"Xfer(D) & Recv(D)in Tostore"]=0

In [230]:
for ind,row in outside_purch.iterrows():
    if row["Xfer(D) & Recv(D)in Tostore"]==0:
        outside_purch.loc[ind, "RMNG PNDG in Fromstore" ]=1
    else:
        outside_purch.loc[ind, "RMNG PNDG in Fromstore" ]=0

In [231]:
outside_purch.columns

Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Qty', 'ItemNetAmount',
       'Outlet Name', 'State', 'City Group', 'Ops Exexutive', 'Cluster',
       'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit', 'T-date',
       'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore'],
      dtype='object')

In [232]:
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Qty,ItemNetAmount,...,start_date,Days,One MDM Code,Unit,T-date,Tra Store id,Total Trasanferd qty,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,2023-10-24,30 days,3-A-CA55-HW-CZ45-AR35-X-00001,10.0,NaN,NaN,0.0,1,0.0,1.0


###### Sales Consolidated For MTD & PM  to track Outside purchased

Dynamics Sales Cleaning

In [233]:
dyn_s.rename(columns={'StyleCode1':"Item Code"},inplace=True)
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox34,Textbox90,Textbox91,Textbox92,Textbox93,Textbox94,Textbox95,Textbox97,Textbox83,Textbox70
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,4.179523e+07,2296054.66,2131123.08,58617.88,0,4805269.63,942598.56,5.747868e+06,10096723.72,10096723.72


In [234]:
dyn_s.columns,dyn_pro.columns

(Index(['Storeid', 'StoreName', 'BillDate', 'BillTime', 'TransType', 'BillNo',
        'Salesman', 'OfferId', 'OfferDescription', 'OriginalBillNo',
        'Item Code', 'ItemDescription', 'ItemReference', 'DrugType',
        'MANUFACTURER', 'Category', 'SubCategory', 'ItemGroup', 'HSNCode',
        'ChronicName', 'IsGenericName', 'Batch', 'ExpiryDate', 'MRP',
        'Quantity', 'MRPValue', 'Value', 'GSV', 'TaxPercent', 'TaxExemted',
        'SGSTValue', 'CGSTValue', 'IGSTValue', 'CESSValue', 'ItemDiscount',
        'BillDiscount', 'TotalDiscount', 'Cost', 'CostWithTax', 'MarginWithTax',
        'MarginWithoutTax', 'Textbox86', 'Textbox26', 'Textbox87', 'Textbox34',
        'Textbox90', 'Textbox91', 'Textbox92', 'Textbox93', 'Textbox94',
        'Textbox95', 'Textbox97', 'Textbox83', 'Textbox70'],
       dtype='object'),
 Index(['Item Code', 'Product name', 'Search name', 'Item Reference',
        'Composition', 'Product type', 'Product subtype',
        'Product dimension groups', 'Pr

In [235]:
dyn_s=pd.merge(dyn_s,dyn_pro[['Item Code','CTG']], on='Item Code', how='left')
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox90,Textbox91,Textbox92,Textbox93,Textbox94,Textbox95,Textbox97,Textbox83,Textbox70,CTG
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,2296054.66,2131123.08,58617.88,0,4805269.63,942598.56,5.747868e+06,10096723.72,10096723.72,FMCG


In [236]:
dyn_s['Storeid'] = dyn_s['Storeid'].astype(str)

In [237]:
dyn_s=pd.merge(dyn_s,sd_details[['Storeid','Outlet Name']], on='Storeid', how='left')
dyn_s.rename(columns={'Store Name As per D 365':"Outlet name"},inplace =True)
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox91,Textbox92,Textbox93,Textbox94,Textbox95,Textbox97,Textbox83,Textbox70,CTG,Outlet Name
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,2131123.08,58617.88,0,4805269.63,942598.56,5.747868e+06,10096723.72,10096723.72,FMCG,KA - Hebbal


In [238]:
dyn_s['Quantity'] = dyn_s['Quantity'].str.replace(',','').astype(float)
dyn_s['MRPValue'] = dyn_s['MRPValue'].str.replace(',','').astype(float)
dyn_s['Value'] = dyn_s['Value'].str.replace(',','').astype(float)
dyn_s['GSV'] = dyn_s['GSV'].str.replace(',','').astype(float)

In [239]:
dyn_s.rename(columns={'Quantity':"Total Qty"},inplace=True)

In [240]:
dyn_s=pd.merge(dyn_s,dyn_pro[['Item Code','One MDM Code']], on='Item Code', how='left')
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox92,Textbox93,Textbox94,Textbox95,Textbox97,Textbox83,Textbox70,CTG,Outlet Name,One MDM Code
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,58617.88,0,4805269.63,942598.56,5.747868e+06,10096723.72,10096723.72,FMCG,KA - Hebbal,3-C-AF2-BL7-EF33-XX09-00007


In [241]:
dyn_s[['DD','MM','YYY']]= dyn_s['BillDate'].str.split("-",expand=True)
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox95,Textbox97,Textbox83,Textbox70,CTG,Outlet Name,One MDM Code,DD,MM,YYY
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,942598.56,5.747868e+06,10096723.72,10096723.72,FMCG,KA - Hebbal,3-C-AF2-BL7-EF33-XX09-00007,01,04,2023


In [242]:
dyn_s['Billdate']=dyn_s['MM'].map(str)+"/"+dyn_s['DD'].map(str)+"/"+dyn_s['YYY']
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox97,Textbox83,Textbox70,CTG,Outlet Name,One MDM Code,DD,MM,YYY,Billdate
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,5.747868e+06,10096723.72,10096723.72,FMCG,KA - Hebbal,3-C-AF2-BL7-EF33-XX09-00007,01,04,2023,04/01/2023


In [243]:
dyn_s['Billdate'] = dyn_s['Billdate'].astype('datetime64[ns]')
dyn_s.head(1)

,Storeid,StoreName,BillDate,BillTime,TransType,BillNo,Salesman,OfferId,OfferDescription,OriginalBillNo,...,Textbox97,Textbox83,Textbox70,CTG,Outlet Name,One MDM Code,DD,MM,YYY,Billdate
0,2901008,KA - Hebbal,01-04-2023,6:58:57,Sales,I290100812200001,FARMAAN SHARIFF,NaN,NaN,NaN,...,5.747868e+06,10096723.72,10096723.72,FMCG,KA - Hebbal,3-C-AF2-BL7-EF33-XX09-00007,01,04,2023,2023-04-01


In [244]:
dyn_rpt=dyn_s.pivot_table(index=['Storeid','Item Code','One MDM Code','Billdate'],values=['Total Qty','MRPValue'],aggfunc='sum')
dyn_rpt.reset_index(inplace=True)
dyn_rpt.rename(columns={'Storeid':"D 365 Store Id",'MRPValue':"Net sale",'BillDate':"Bill Date",'BillNo':"Bill No."},inplace=True)
print(dyn_rpt.shape)
dyn_rpt.head()

(2016030, 6)


,D 365 Store Id,Item Code,One MDM Code,Billdate,Net sale,Total Qty
0,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-07-07,26.00,2.0
1,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-08-08,39.00,3.0
2,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-10-07,65.00,5.0
3,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-10-08,117.00,9.0
4,2901001,P-0000003,3-A-AS03-HW-BF13-XX09-X-00002,2023-07-29,11.16,1.0


In [245]:
uniq_master.rename(columns={'Pro Code':"Item Code",'MDM Code':"One MDM Code"},inplace=True)
uniq_master.columns
uniq_master['Item Code']=uniq_master['Item Code'].astype(float)

In [246]:
gof_s= pd.merge(gof_s,uniq_master[['Item Code','One MDM Code']],on='Item Code',how='left')
gof_s.head(1)

,Outlet Name,Bill No.,Bill Date,Item Code,Item Name,Phone1,Mobile Number & SMS alerts,Mobile2,Telephone,CUST GST NO,...,Net Cost (With Vat),VatAmt (Purchase),Net Cost (Without Vat),Margin On NS(WOT Vat),Margin On NS(With Vat),Margin%,Landing Cost,MRP,Item Rate,One MDM Code
0,AP - Currency Nagar,CA1838,01-04-2023,110048,KOFLET LOZENGES JAR 200S,,9515370679,,9515370679,,...,7.38,0.78,6.6,1.97,2.22,22.99,1.23,1.6,1.6,3-K-BM4-EO4-CK76-XX09-00029


In [247]:
gof_s=pd.merge(gof_s,sd_details[['Outlet Name','Storeid','Gof_status']],on='Outlet Name',how='left')
gof_s.head(1)

,Outlet Name,Bill No.,Bill Date,Item Code,Item Name,Phone1,Mobile Number & SMS alerts,Mobile2,Telephone,CUST GST NO,...,Net Cost (Without Vat),Margin On NS(WOT Vat),Margin On NS(With Vat),Margin%,Landing Cost,MRP,Item Rate,One MDM Code,Storeid,Gof_status
0,AP - Currency Nagar,CA1838,01-04-2023,110048,KOFLET LOZENGES JAR 200S,,9515370679,,9515370679,,...,6.6,1.97,2.22,22.99,1.23,1.6,1.6,3-K-BM4-EO4-CK76-XX09-00029,3701001,In-Active


In [248]:
sd_details.columns

Index(['S. No.', 'Store Code', 'Store Type', 'HDH Store Code', 'HPD BU2 ID',
       'HPD TNG', 'Get Well', 'SreeJi', 'Vasu agency', 'Venkat Sai',
       'HPD Kerala', 'PO Store Name', 'Store NameAs per Branding ',
       'Outlet Name', 'GOF Store Id', 'Launch Date', 'Email ID', 'Short Name',
       'Build-up Area in Sft', 'Phone Number', 'Ops Exexutive',
       'Cluster Mail Id', 'Executive Phone Number', 'Area Manager',
       'Sub Region', 'State', 'City', 'City 2', 'Gof_status', 'Manager',
       'Cluster', 'Any Desk ID', 'Rental', 'FF Target', 'Address', 'Zip Code',
       'Region', 'HOO', 'Outlet Format', 'Tenure Days', 'Tenure Month',
       'Tenure Years', 'Tenure', 'Launch Month', 'Launch Year', 'Quarter',
       'Financial Year', 'Securty Deposit', 'Rent Start Date', 'Store System',
       'Storeid', 'Store Name As per D 365', 'Retail Channel Id', 'Party ID',
       'Fin And Qtr', 'Location Series', 'City G', 'Vendor Code'],
      dtype='object')

In [249]:
gof_s[['DD','MM','YYY']]=gof_s['Bill Date'].str.split('-',expand=True)

In [250]:
gof_s['Billdate']=gof_s['MM'].map(str)+"/"+gof_s['DD'].map(str)+"/"+gof_s['YYY']

In [251]:
gof_s['Billdate'] = gof_s['Billdate'].astype('datetime64[ns]')

In [252]:
gof_s.head(1)

,Outlet Name,Bill No.,Bill Date,Item Code,Item Name,Phone1,Mobile Number & SMS alerts,Mobile2,Telephone,CUST GST NO,...,Landing Cost,MRP,Item Rate,One MDM Code,Storeid,Gof_status,DD,MM,YYY,Billdate
0,AP - Currency Nagar,CA1838,01-04-2023,110048,KOFLET LOZENGES JAR 200S,,9515370679,,9515370679,,...,1.23,1.6,1.6,3-K-BM4-EO4-CK76-XX09-00029,3701001,In-Active,01,04,2023,2023-04-01


In [253]:
gof_s['Qty'] = gof_s['Qty'].str.replace(',','').astype(float)
gof_s['Net Sales Less Disc'] = gof_s['Net Sales Less Disc'].str.replace(',','').astype(float)
gof_s['Net Sales (With Vat)'] = gof_s['Net Sales (With Vat)'].str.replace(',','').astype(float)

In [254]:
gof_rpt=gof_s.pivot_table(index=['Storeid',
                                         'One MDM Code','Item Code','Billdate'],values=['Qty','Net Sales (With Vat)','Free Qty'],aggfunc='sum')
gof_rpt.reset_index(inplace=True)
gof_rpt.rename(columns={'Qty':"Total Qty",'Net Sales (With Vat)':"Net sale"},inplace=True)
print(gof_rpt.shape)
gof_rpt.head()

(1605874, 7)


,Storeid,One MDM Code,Item Code,Billdate,Free Qty,Net sale,Total Qty
0,3201005,3-A-AA22-HW-FM28-XX09-X-00001,350004,2023-09-19,0,384.0,24.0
1,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002,2023-04-06,0,810.0,90.0
2,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002,2023-04-10,0,540.0,60.0
3,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002,2023-04-22,0,360.0,40.0
4,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002,2023-04-25,0,180.0,20.0


In [255]:
dyn_rpt.rename(columns={'D 365 Store Id':"Storeid",'Item Code':"Item Code(D 365)"},inplace=True)
gof_rpt.rename(columns={'Item Code':"Item Code(Gofrugal)"},inplace=True)
dyn_rpt.head()

,Storeid,Item Code(D 365),One MDM Code,Billdate,Net sale,Total Qty
0,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-07-07,26.00,2.0
1,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-08-08,39.00,3.0
2,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-10-07,65.00,5.0
3,2901001,P-0000002,3-A-AY42-HW-BB17-XX09-X-00001,2023-10-08,117.00,9.0
4,2901001,P-0000003,3-A-AS03-HW-BF13-XX09-X-00002,2023-07-29,11.16,1.0


In [256]:
print('Gofrugal :',gof_rpt.shape,'','Dynamics :',dyn_rpt.shape)
merg_sales_rpt = pd.concat([gof_rpt,dyn_rpt])
print(merg_sales_rpt.shape)
merg_sales_rpt.head(3)

Gofrugal : (1605874, 7)  Dynamics : (2016030, 6)
(3621904, 8)


,Storeid,One MDM Code,Item Code(Gofrugal),Billdate,Free Qty,Net sale,Total Qty,Item Code(D 365)
0,3201005,3-A-AA22-HW-FM28-XX09-X-00001,350004.0,2023-09-19,0.0,384.0,24.0,NaN
1,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-06,0.0,810.0,90.0,NaN
2,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-10,0.0,540.0,60.0,NaN


In [257]:
merg_sales_rpt.reset_index(inplace=True)
merg_sales_rpt.drop('index',axis=1,inplace=True)
merg_sales_rpt.head()

,Storeid,One MDM Code,Item Code(Gofrugal),Billdate,Free Qty,Net sale,Total Qty,Item Code(D 365)
0,3201005,3-A-AA22-HW-FM28-XX09-X-00001,350004.0,2023-09-19,0.0,384.0,24.0,NaN
1,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-06,0.0,810.0,90.0,NaN
2,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-10,0.0,540.0,60.0,NaN
3,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-22,0.0,360.0,40.0,NaN
4,3201005,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-25,0.0,180.0,20.0,NaN


In [258]:
merg_sales_rpt['Storeid']=merg_sales_rpt['Storeid'].astype(float)

In [259]:
merg_sales_rpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3621904 entries, 0 to 3621903
Data columns (total 8 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Storeid              float64       
 1   One MDM Code         object        
 2   Item Code(Gofrugal)  float64       
 3   Billdate             datetime64[ns]
 4   Free Qty             float64       
 5   Net sale             float64       
 6   Total Qty            float64       
 7   Item Code(D 365)     object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 221.1+ MB


In [260]:
outside_purch.rename(columns={'Qty':"Pur Qty In strips"},inplace=True)

In [261]:
outside_purch["Purchase Qty"]=outside_purch["Pur Qty In strips"]*outside_purch["Unit"]

In [262]:
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Pur Qty In strips,ItemNetAmount,...,Days,One MDM Code,Unit,T-date,Tra Store id,Total Trasanferd qty,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore,Purchase Qty
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,30 days,3-A-CA55-HW-CZ45-AR35-X-00001,10.0,NaN,NaN,0.0,1,0.0,1.0,20.0


###### Transferd store & sales mapping  data 

where It will Create duplicates Lines

In [263]:
ibt_sales_merge=merg_sales_rpt.copy()
print(ibt_sales_merge.shape)

(3621904, 8)


In [264]:
ibt_to_sc.head()

,Storeid,TransDate,ToStore,Item Code,Purchase Date,Total Trasanferd qty,Tra Store id,T-date
0,2901001,2023-10-05,2901076,P-0014904,2023-10-05,1.0,2901076|,"2023-10-05,"
1,2907004,2023-10-06,2907003,P-0032947,2023-10-06,60.0,2907003|,"2023-10-06,"
2,2907004,2023-10-06,2907003,P-0040862,2023-10-06,3.0,2907003|,"2023-10-06,"
3,2907004,2023-10-06,2907003,P-0087159,2023-10-06,30.0,2907003|,"2023-10-06,"
4,2907004,2023-10-06,2907003,P-0087737,2023-10-06,30.0,2907003|,"2023-10-06,"


In [265]:
ibt_sales_merge.rename(columns={'Storeid':"ToStore"},inplace=True)

In [266]:
ibt_sales_merge.head()

,ToStore,One MDM Code,Item Code(Gofrugal),Billdate,Free Qty,Net sale,Total Qty,Item Code(D 365)
0,3201005.0,3-A-AA22-HW-FM28-XX09-X-00001,350004.0,2023-09-19,0.0,384.0,24.0,NaN
1,3201005.0,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-06,0.0,810.0,90.0,NaN
2,3201005.0,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-10,0.0,540.0,60.0,NaN
3,3201005.0,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-22,0.0,360.0,40.0,NaN
4,3201005.0,3-A-AA23-HW-AS45-BE05-X-00001,167002.0,2023-04-25,0.0,180.0,20.0,NaN


In [267]:
ibt_sales_merge['Item Code(Gofrugal)'].fillna(0,inplace=True)

In [268]:
ibt_sales_merge['Item Code(Gofrugal)']=ibt_sales_merge['Item Code(Gofrugal)'].astype(int).astype(str)

In [269]:
ibt_sales_merge['Item Code(Gofrugal)'].dtype

dtype('O')

In [270]:
outside_purch.columns

Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Pur Qty In strips',
       'ItemNetAmount', 'Outlet Name', 'State', 'City Group', 'Ops Exexutive',
       'Cluster', 'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit',
       'T-date', 'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore',
       'Purchase Qty'],
      dtype='object')

In [271]:
ibt_sales_merge['Item Code']=ibt_sales_merge['Item Code(D 365)']

In [272]:
ibt_sales_merge['Item Code'].fillna(ibt_sales_merge['Item Code(Gofrugal)'],inplace=True)

In [273]:
print(ibt_sales_merge.shape)
#sales_merger.to_csv("sales_merger.csv")

(3621904, 9)


In [274]:
#outside_purch["Storeid"]=outside_purch["Storeid"].astype(str)
ibt_sales_merge["ToStore"].dtypes

dtype('float64')

In [275]:
ibt_sales_merge["ToStore"]=ibt_sales_merge["ToStore"].astype(int).astype(str)

In [276]:
ibt_sales_merge["ToStore"].dtypes

dtype('O')

In [277]:
#sales_merger=pd.merge(sales_merger,outside_purch,on=['Storeid','One MDM Code'])
ibt_sales_merge=pd.merge(ibt_sales_merge,ibt_to_sc,on=['ToStore','Item Code'])

In [278]:
print(ibt_sales_merge.shape)
ibt_sales_merge.head(1)

(1321, 15)


,ToStore,One MDM Code,Item Code(Gofrugal),Billdate,Free Qty,Net sale,Total Qty,Item Code(D 365),Item Code,Storeid,TransDate,Purchase Date,Total Trasanferd qty,Tra Store id,T-date
0,3202001,3-A-AM48-HW-DO08-AW50-X-00013,916386,2023-04-23,0.0,23.42,3.0,NaN,916386,3202003,2023-10-09,2023-10-03,20.0,3202001|,"2023-10-09,"


In [279]:
for ind,row in ibt_sales_merge.iterrows():
    if row['Billdate'] <row['TransDate']:
        ibt_sales_merge.loc[ind,'After Transfer sold Qty']=0
    else:ibt_sales_merge.loc[ind,'After Transfer sold Qty']=row['Total Qty']

In [280]:
ibt_sales_merge.columns

Index(['ToStore', 'One MDM Code', 'Item Code(Gofrugal)', 'Billdate',
       'Free Qty', 'Net sale', 'Total Qty', 'Item Code(D 365)', 'Item Code',
       'Storeid', 'TransDate', 'Purchase Date', 'Total Trasanferd qty',
       'Tra Store id', 'T-date', 'After Transfer sold Qty'],
      dtype='object')

In [281]:
#sm_look=sales_merger.pivot_table(index=['Storeid', 'One MDM Code','VendName','Purchase Date'],values=['After Purchase sold Qty'],aggfunc='sum')
ibt_sm_look=ibt_sales_merge.pivot_table(index=['Storeid', 'Item Code','Purchase Date'],values=['After Transfer sold Qty'],aggfunc='sum')

In [282]:
ibt_sm_look.reset_index(inplace=True)

In [283]:
ibt_sm_look.head()

,Storeid,Item Code,Purchase Date,After Transfer sold Qty
0,2907004,P-0032947,2023-10-06,60.0
1,2907004,P-0040862,2023-10-06,3.0
2,2907004,P-0087159,2023-10-06,30.0
3,2907004,P-0087737,2023-10-06,30.0
4,3202002,229010,2023-10-24,0.0


In [284]:
outside_purch.shape

(4857, 27)

In [285]:
#outside_purch=pd.merge(outside_purch,sm_look[['Storeid', 'One MDM Code','Purchase Date','VendName','After Purchase sold Qty']],on=['Storeid', 'One MDM Code','Purchase Date','VendName'],how='left')
outside_purch=pd.merge(outside_purch,ibt_sm_look[['Storeid', 'Item Code','Purchase Date','After Transfer sold Qty']],on=['Storeid', 'Item Code','Purchase Date'],how='left')

In [286]:
outside_purch.shape

(4857, 28)

In [287]:
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Pur Qty In strips,ItemNetAmount,...,One MDM Code,Unit,T-date,Tra Store id,Total Trasanferd qty,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore,Purchase Qty,After Transfer sold Qty
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,3-A-CA55-HW-CZ45-AR35-X-00001,10.0,NaN,NaN,0.0,1,0.0,1.0,20.0,NaN


In [288]:
outside_purch['After Transfer sold Qty'].fillna(0,inplace=True)

###### Op Purchase & sales mapping  data 

where It will Create duplicates Lines

In [289]:
sales_merger=merg_sales_rpt.copy()
print(sales_merger.shape)

(3621904, 8)


In [290]:
sales_merger.columns

Index(['Storeid', 'One MDM Code', 'Item Code(Gofrugal)', 'Billdate',
       'Free Qty', 'Net sale', 'Total Qty', 'Item Code(D 365)'],
      dtype='object')

In [291]:
sales_merger['Item Code(Gofrugal)'].fillna(0,inplace=True)

In [292]:
sales_merger['Item Code(Gofrugal)']=sales_merger['Item Code(Gofrugal)'].astype(int).astype(str)

In [293]:
sales_merger['Item Code(Gofrugal)'].dtype

dtype('O')

In [294]:
outside_purch.columns

Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Pur Qty In strips',
       'ItemNetAmount', 'Outlet Name', 'State', 'City Group', 'Ops Exexutive',
       'Cluster', 'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit',
       'T-date', 'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore', 'Purchase Qty',
       'After Transfer sold Qty'],
      dtype='object')

In [295]:
sales_merger['Item Code']=sales_merger['Item Code(D 365)']

In [296]:
sales_merger['Item Code'].fillna(sales_merger['Item Code(Gofrugal)'],inplace=True)

In [297]:
print(sales_merger.shape)
#sales_merger.to_csv("sales_merger.csv")

(3621904, 9)


In [298]:
sales_merger['Storeid']=sales_merger['Storeid'].astype(int).astype(str)

In [299]:
#sales_merger=pd.merge(sales_merger,outside_purch,on=['Storeid','One MDM Code'])
sales_merger=pd.merge(sales_merger,outside_purch,on=['Storeid','Item Code'])

In [300]:
sales_merger.columns

Index(['Storeid', 'One MDM Code_x', 'Item Code(Gofrugal)', 'Billdate',
       'Free Qty', 'Net sale', 'Total Qty', 'Item Code(D 365)', 'Item Code',
       'Purchase Date', 'StoreName', 'VendName', 'ItemName', 'Vendor Group',
       'CTG PRO', 'Pur Qty In strips', 'ItemNetAmount', 'Outlet Name', 'State',
       'City Group', 'Ops Exexutive', 'Cluster', 'Store Type', 'start_date',
       'Days', 'One MDM Code_y', 'Unit', 'T-date', 'Tra Store id',
       'Total Trasanferd qty', 'Overall', 'Xfer(D) & Recv(D)in Tostore',
       'RMNG PNDG in Fromstore', 'Purchase Qty', 'After Transfer sold Qty'],
      dtype='object')

In [301]:
print(sales_merger.shape)
sales_merger.head(1)

(44817, 35)


,Storeid,One MDM Code_x,Item Code(Gofrugal),Billdate,Free Qty,Net sale,Total Qty,Item Code(D 365),Item Code,Purchase Date,...,One MDM Code_y,Unit,T-date,Tra Store id,Total Trasanferd qty,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore,Purchase Qty,After Transfer sold Qty
0,3201008,3-A-AB52-AD-EF33-BU18-X-00051,537064,2023-04-16,0.0,170.0,1.0,NaN,537064,2023-10-04,...,3-A-AB52-AD-EF33-BU18-X-00051,1.0,NaN,NaN,0.0,1,0.0,1.0,2.0,0.0


In [302]:
for ind,row in sales_merger.iterrows():
    if row['Billdate'] <row['Purchase Date']:
        sales_merger.loc[ind,'After Purchase sold Qty']=0
    else:sales_merger.loc[ind,'After Purchase sold Qty']=row['Total Qty']

In [303]:
sales_merger.columns

Index(['Storeid', 'One MDM Code_x', 'Item Code(Gofrugal)', 'Billdate',
       'Free Qty', 'Net sale', 'Total Qty', 'Item Code(D 365)', 'Item Code',
       'Purchase Date', 'StoreName', 'VendName', 'ItemName', 'Vendor Group',
       'CTG PRO', 'Pur Qty In strips', 'ItemNetAmount', 'Outlet Name', 'State',
       'City Group', 'Ops Exexutive', 'Cluster', 'Store Type', 'start_date',
       'Days', 'One MDM Code_y', 'Unit', 'T-date', 'Tra Store id',
       'Total Trasanferd qty', 'Overall', 'Xfer(D) & Recv(D)in Tostore',
       'RMNG PNDG in Fromstore', 'Purchase Qty', 'After Transfer sold Qty',
       'After Purchase sold Qty'],
      dtype='object')

In [304]:
#sm_look=sales_merger.pivot_table(index=['Storeid', 'One MDM Code','VendName','Purchase Date'],values=['After Purchase sold Qty'],aggfunc='sum')
sm_look=sales_merger.pivot_table(index=['Storeid', 'Item Code','VendName','Purchase Date'],values=['After Purchase sold Qty'],aggfunc='sum')

In [305]:
sm_look.reset_index(inplace=True)

In [306]:
sm_look.head()

,Storeid,Item Code,VendName,Purchase Date,After Purchase sold Qty
0,2901001,P-0002566,Local Vendor Karnataka,2023-09-27,60.0
1,2901001,P-0014904,Local Vendor Karnataka,2023-10-05,1.0
2,2901001,P-0025509,Local Vendor Karnataka,2023-10-11,1.0
3,2901001,P-0030457,Local Vendor Karnataka,2023-10-17,4.0
4,2901040,P-0089682,DHAVANTHRI SPECIALITY PHARMA PVT LTD,2023-10-15,56.0


In [307]:
outside_purch.shape

(4857, 28)

In [308]:
#outside_purch=pd.merge(outside_purch,sm_look[['Storeid', 'One MDM Code','Purchase Date','VendName','After Purchase sold Qty']],on=['Storeid', 'One MDM Code','Purchase Date','VendName'],how='left')
outside_purch=pd.merge(outside_purch,sm_look[['Storeid', 'Item Code','Purchase Date','VendName','After Purchase sold Qty']],on=['Storeid', 'Item Code','Purchase Date','VendName'],how='left')

In [309]:
outside_purch['After Purchase sold Qty'].fillna(0,inplace=True)

In [310]:
outside_purch.loc[outside_purch['After Purchase sold Qty'] <0 , 'After Purchase sold Qty'] = 0

In [311]:
outside_purch['Total Sold Qty']=outside_purch['After Transfer sold Qty']+outside_purch['After Purchase sold Qty']

In [312]:
outside_purch.shape

(4857, 30)

In [313]:
for ind,row in outside_purch.iterrows():
    if row['Total Sold Qty'] >row['Purchase Qty']:
        outside_purch.loc[ind,'Con Sold Qty']=row['Purchase Qty']
    else:outside_purch.loc[ind,'Con Sold Qty']=row['Total Sold Qty']

In [314]:
outside_purch.rename(columns={"Con Sold Qty":"Sold Qty",'ItemNetAmount':"OP Value"},inplace=True)
print(outside_purch.shape)
outside_purch.columns

(4857, 31)


Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG PRO', 'Pur Qty In strips', 'OP Value',
       'Outlet Name', 'State', 'City Group', 'Ops Exexutive', 'Cluster',
       'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit', 'T-date',
       'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore', 'Purchase Qty',
       'After Transfer sold Qty', 'After Purchase sold Qty', 'Total Sold Qty',
       'Sold Qty'],
      dtype='object')

In [315]:
outside_purch['Converted Value']=(outside_purch['OP Value']/outside_purch['Purchase Qty'])*outside_purch['Sold Qty']

In [316]:
outside_purch.head(1)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Pur Qty In strips,OP Value,...,Total Trasanferd qty,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore,Purchase Qty,After Transfer sold Qty,After Purchase sold Qty,Total Sold Qty,Sold Qty,Converted Value
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,0.0,1,0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0


In [317]:
for ind,row in outside_purch.iterrows():
    if row['Sold Qty']<=0:
        outside_purch.loc[ind,'Sales Status']='Not Converted'
    else:
        outside_purch.loc[ind,'Sales Status']='Converted'

In [318]:
print(outside_purch.shape)
outside_purch.head(2)

(4857, 33)


,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG PRO,Pur Qty In strips,OP Value,...,Overall,Xfer(D) & Recv(D)in Tostore,RMNG PNDG in Fromstore,Purchase Qty,After Transfer sold Qty,After Purchase sold Qty,Total Sold Qty,Sold Qty,Converted Value,Sales Status
0,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0020495,TOLMOVE 150 TABLET,Outside Purchase,PHARMA,2.0,221.76,...,1,0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.00,Not Converted
1,2023-09-24,3201021,KL - Sub Jail Road Aluva,Local Vendor Kerala,P-0024340,WAKCERIN GM TABLET 10S,Outside Purchase,PHARMA,1.0,140.91,...,1,0.0,1.0,10.0,0.0,15.0,15.0,10.0,140.91,Converted


In [319]:
for ind,row in outside_purch.iterrows():
    if row['After Transfer sold Qty']>0:
        outside_purch.loc[ind,'Conv on Recv(D)store']=1
    else:
        outside_purch.loc[ind,'Conv on Recv(D)store']=0

In [320]:
for ind,row in outside_purch.iterrows():
    if row['Sold Qty']<=0:
        outside_purch.loc[ind,'Sales Biferication']='Not Converted'
    elif row['Sold Qty']>=row['Purchase Qty']:
        outside_purch.loc[ind,'Sales Biferication']='Converted'
    else:
        outside_purch.loc[ind,'Sales Biferication']='Partially Converted'

In [321]:
for ind,row in outside_purch.iterrows():
    if row['Sales Status']=='Converted':
        outside_purch.loc[ind,'Converted']=1
    else:
        outside_purch.loc[ind,'Converted']=0

In [322]:
for ind,row in outside_purch.iterrows():
    if row['Sales Biferication']=='Converted':
        outside_purch.loc[ind,'Fully Converted']=1
    else:
        outside_purch.loc[ind,'Fully Converted']=0

In [323]:
for ind,row in outside_purch.iterrows():
    if row['Sales Biferication']=='Partially Converted':
        outside_purch.loc[ind,'Partially Converted']=1
    else:
        outside_purch.loc[ind,'Partially Converted']=0

In [324]:
outside_purch.rename(columns={"CTG PRO":"CTG","Fully Converted":"Fully Conv",'Partially Converted':"Partial Conv"},inplace=True)
outside_purch.columns

Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG', 'Pur Qty In strips', 'OP Value',
       'Outlet Name', 'State', 'City Group', 'Ops Exexutive', 'Cluster',
       'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit', 'T-date',
       'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore', 'Purchase Qty',
       'After Transfer sold Qty', 'After Purchase sold Qty', 'Total Sold Qty',
       'Sold Qty', 'Converted Value', 'Sales Status', 'Conv on Recv(D)store',
       'Sales Biferication', 'Converted', 'Fully Conv', 'Partial Conv'],
      dtype='object')

In [325]:
valid=outside_purch["Converted"].sum(),outside_purch["Fully Conv"].sum(),outside_purch["Partial Conv"].sum()

In [326]:
state_sum7d=outside_purch[outside_purch["Days"]<="7 Days"].pivot_table(index=["State","CTG"],values=["Overall"
                                                                                                     ,"Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum",margins=True,margins_name="Total")
state_sum7d.reset_index(inplace=True)

In [327]:
state_sum7d_st=outside_purch[outside_purch["Days"]<="7 Days"].pivot_table(index=["State",],values=["Overall"
                                                                                                     ,"Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum")
state_sum7d_st.reset_index(inplace=True)

In [328]:
state_sum7d_st['CTG']="Total"

In [329]:
state_sum7d_st

,State,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG
0,Karnataka,2.0,2275.840000,2.0,2275.84,2,0.0,Total
1,Kerala,510.0,208080.797698,319.0,323977.96,738,191.0,Total
2,Telangana,107.0,79650.605083,101.0,99632.06,147,6.0,Total


In [330]:
state_sum7d = pd.concat([state_sum7d, state_sum7d_st], sort=False).sort_values(['State', 'CTG'])

In [331]:
state_sum7d.head()

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,PHARMA,2.0,2275.840000,2.0,2275.84,2,0.0
0,Karnataka,Total,2.0,2275.840000,2.0,2275.84,2,0.0
1,Kerala,NON -PHARMA,20.0,11870.972064,13.0,13688.40,28,7.0
2,Kerala,PHARMA,490.0,196209.825634,306.0,310289.56,710,184.0
1,Kerala,Total,510.0,208080.797698,319.0,323977.96,738,191.0


In [332]:
state_sum7d_col_int = state_sum7d.select_dtypes(include=['int', 'float']).columns
state_sum7d[state_sum7d_col_int]=state_sum7d[state_sum7d_col_int].astype(int)
state_sum7d.head(3)

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,PHARMA,2,2275,2,2275,2,0
0,Karnataka,Total,2,2275,2,2275,2,0
1,Kerala,NON -PHARMA,20,11870,13,13688,28,7


In [333]:
state_sum7d["FC %"]=state_sum7d["Fully Conv"]/state_sum7d["Overall"]
state_sum7d["PC %"]=state_sum7d["Partial Conv"]/state_sum7d["Overall"]
state_sum7d["Sku Con %"]=state_sum7d["Converted"]/state_sum7d["Overall"]
state_sum7d["Sales Con %"]=state_sum7d["Converted Value"]/state_sum7d["OP Value"]

In [334]:
state_sum7d

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,FC %,PC %,Sku Con %,Sales Con %
0,Karnataka,PHARMA,2,2275,2,2275,2,0,1.000000,0.000000,1.000000,1.000000
0,Karnataka,Total,2,2275,2,2275,2,0,1.000000,0.000000,1.000000,1.000000
1,Kerala,NON -PHARMA,20,11870,13,13688,28,7,0.464286,0.250000,0.714286,0.867183
2,Kerala,PHARMA,490,196209,306,310289,710,184,0.430986,0.259155,0.690141,0.632343
1,Kerala,Total,510,208080,319,323977,738,191,0.432249,0.258808,0.691057,0.642268
3,Telangana,NON -PHARMA,3,2935,2,5338,6,1,0.333333,0.166667,0.500000,0.549831
4,Telangana,PHARMA,104,76715,99,94293,141,5,0.702128,0.035461,0.737589,0.813581
2,Telangana,Total,107,79650,101,99632,147,6,0.687075,0.040816,0.727891,0.799442
5,Total,,619,290007,422,425885,886,197,0.476298,0.222348,0.698646,0.680951


In [335]:
state_sum7d=state_sum7d.loc[:,["State",'CTG','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [336]:
state_sum7d.head()

,State,CTG,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
0,Karnataka,PHARMA,2,2,1.000000,0,0.000000,2,1.000000,2275,2275,1.000000
0,Karnataka,Total,2,2,1.000000,0,0.000000,2,1.000000,2275,2275,1.000000
1,Kerala,NON -PHARMA,28,13,0.464286,7,0.250000,20,0.714286,13688,11870,0.867183
2,Kerala,PHARMA,710,306,0.430986,184,0.259155,490,0.690141,310289,196209,0.632343
1,Kerala,Total,738,319,0.432249,191,0.258808,510,0.691057,323977,208080,0.642268


In [337]:
opsexe_sum7d=outside_purch[outside_purch["Days"]<="7 Days"].pivot_table(index=["State","City Group",'Ops Exexutive'],values=["Overall"
                                                                                                     ,"Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum",margins=True,margins_name="Total")
opsexe_sum7d.reset_index(inplace=True)

In [338]:
opsexe_sum7d.head()

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,KA,Byregowda MR,2.0,2275.840000,2.0,2275.84,2,0.0
1,Kerala,KL,Akshai Chandran,4.0,794.286000,3.0,2032.85,5,1.0
2,Kerala,KL,Fathima Rukzana,8.0,12077.668333,7.0,13138.29,11,1.0
3,Kerala,ROKL,Asha Sk,141.0,44058.752897,85.0,55701.38,154,56.0
4,Kerala,ROKL,Girish Kumar,239.0,76595.633251,135.0,142811.25,402,104.0


In [339]:
state_sum7d_st["Ops Exexutive"]="Total"
state_sum7d_st["City Group"]="-"

In [340]:
opsexe_sum7d = pd.concat([opsexe_sum7d, state_sum7d_st], sort=False).sort_values(['State', 'Ops Exexutive'])

In [341]:
opsexe_sum7d.head()

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG
0,Karnataka,KA,Byregowda MR,2.0,2275.840000,2.0,2275.84,2,0.0,NaN
0,Karnataka,-,Total,2.0,2275.840000,2.0,2275.84,2,0.0,Total
1,Kerala,KL,Akshai Chandran,4.0,794.286000,3.0,2032.85,5,1.0,NaN
3,Kerala,ROKL,Asha Sk,141.0,44058.752897,85.0,55701.38,154,56.0,NaN
2,Kerala,KL,Fathima Rukzana,8.0,12077.668333,7.0,13138.29,11,1.0,NaN


In [342]:
opsexe_sum7d_col_int = opsexe_sum7d.select_dtypes(include=['int', 'float']).columns
opsexe_sum7d[opsexe_sum7d_col_int]=opsexe_sum7d[opsexe_sum7d_col_int].astype(int)
opsexe_sum7d.head(3)

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG
0,Karnataka,KA,Byregowda MR,2,2275,2,2275,2,0,NaN
0,Karnataka,-,Total,2,2275,2,2275,2,0,Total
1,Kerala,KL,Akshai Chandran,4,794,3,2032,5,1,NaN


In [343]:
opsexe_sum7d["FC %"]=opsexe_sum7d["Fully Conv"]/opsexe_sum7d["Overall"]
opsexe_sum7d["PC %"]=opsexe_sum7d["Partial Conv"]/opsexe_sum7d["Overall"]
opsexe_sum7d["Sku Con %"]=opsexe_sum7d["Converted"]/opsexe_sum7d["Overall"]
opsexe_sum7d["Sales Con %"]=opsexe_sum7d["Converted Value"]/opsexe_sum7d["OP Value"]

In [344]:
opsexe_sum7d=opsexe_sum7d.loc[:,["State",'City Group','Ops Exexutive','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [345]:
opsexe_sum7d.reset_index(drop=True,inplace=True)
opsexe_sum7d

,State,City Group,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
0,Karnataka,KA,Byregowda MR,2,2,1.000000,0,0.000000,2,1.000000,2275,2275,1.000000
1,Karnataka,-,Total,2,2,1.000000,0,0.000000,2,1.000000,2275,2275,1.000000
2,Kerala,KL,Akshai Chandran,5,3,0.600000,1,0.200000,4,0.800000,2032,794,0.390748
3,Kerala,ROKL,Asha Sk,154,85,0.551948,56,0.363636,141,0.915584,55701,44058,0.790973
4,Kerala,KL,Fathima Rukzana,11,7,0.636364,1,0.090909,8,0.727273,13138,12077,0.919242
5,Kerala,ROKL,Girish Kumar,402,135,0.335821,104,0.258706,239,0.594527,142811,76595,0.536338
6,Kerala,ROKL,Jithin M,3,3,1.000000,0,0.000000,3,1.000000,9581,9581,1.000000
7,Kerala,ROKL,Prejith Mt,163,86,0.527607,29,0.177914,115,0.705521,100712,64972,0.645127
8,Kerala,-,Total,738,319,0.432249,191,0.258808,510,0.691057,323977,208080,0.642268
9,Telangana,TG,Ephraim,2,2,1.000000,0,0.000000,2,1.000000,627,627,1.000000


In [346]:
state_summary=outside_purch.pivot_table(index=["State","CTG"],values=["Overall","Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum",margins=True,margins_name="Total")
state_summary.reset_index(inplace=True)

In [347]:
state_summary

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,PHARMA,18.0,1.030119e+05,18.0,103538.96,20,0.0
1,Kerala,NON -PHARMA,175.0,1.077684e+05,138.0,149833.28,254,37.0
2,Kerala,PHARMA,2638.0,1.183125e+06,1851.0,1629793.86,3490,787.0
3,Kerala,PRIVATE_LABEL,6.0,6.227740e+02,0.0,3836.62,10,6.0
4,Telangana,NON -PHARMA,51.0,4.025540e+04,44.0,54113.50,61,7.0
5,Telangana,PHARMA,850.0,4.595949e+05,808.0,542468.65,1022,42.0
6,Total,,3738.0,1.894379e+06,2859.0,2483584.87,4854,879.0


In [348]:
state_summary_st=outside_purch.pivot_table(index=["State",],values=["Overall" ,"Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum")
state_summary_st.reset_index(inplace=True)
state_summary_st['CTG']="Total"
state_summary = pd.concat([state_summary, state_summary_st], sort=False).sort_values(['State', 'CTG'])

In [349]:
state_summary

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,PHARMA,18.0,1.030119e+05,18.0,103538.96,20,0.0
0,Karnataka,Total,18.0,1.030119e+05,18.0,103538.96,20,0.0
1,Kerala,NON -PHARMA,175.0,1.077684e+05,138.0,149833.28,254,37.0
2,Kerala,PHARMA,2638.0,1.183125e+06,1851.0,1629793.86,3490,787.0
3,Kerala,PRIVATE_LABEL,6.0,6.227740e+02,0.0,3836.62,10,6.0
1,Kerala,Total,2819.0,1.291517e+06,1989.0,1783463.76,3754,830.0
4,Telangana,NON -PHARMA,51.0,4.025540e+04,44.0,54113.50,61,7.0
5,Telangana,PHARMA,850.0,4.595949e+05,808.0,542468.65,1022,42.0
2,Telangana,Total,901.0,4.998503e+05,852.0,596582.15,1083,49.0
6,Total,,3738.0,1.894379e+06,2859.0,2483584.87,4854,879.0


In [350]:
state_summary_col_int = state_summary.select_dtypes(include=['int', 'float']).columns
state_summary[state_summary_col_int]=state_summary[state_summary_col_int].astype(int)
state_summary.head(3)

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,PHARMA,18,103011,18,103538,20,0
0,Karnataka,Total,18,103011,18,103538,20,0
1,Kerala,NON -PHARMA,175,107768,138,149833,254,37


In [351]:
state_summary["FC %"]=state_summary["Fully Conv"]/state_summary["Overall"]
state_summary["PC %"]=state_summary["Partial Conv"]/state_summary["Overall"]
state_summary["Sku Con %"]=state_summary["Converted"]/state_summary["Overall"]
state_summary["Sales Con %"]=state_summary["Converted Value"]/state_summary["OP Value"]

In [352]:
state_summary

,State,CTG,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,FC %,PC %,Sku Con %,Sales Con %
0,Karnataka,PHARMA,18,103011,18,103538,20,0,0.900000,0.000000,0.900000,0.994910
0,Karnataka,Total,18,103011,18,103538,20,0,0.900000,0.000000,0.900000,0.994910
1,Kerala,NON -PHARMA,175,107768,138,149833,254,37,0.543307,0.145669,0.688976,0.719254
2,Kerala,PHARMA,2638,1183125,1851,1629793,3490,787,0.530372,0.225501,0.755874,0.725936
3,Kerala,PRIVATE_LABEL,6,622,0,3836,10,6,0.000000,0.600000,0.600000,0.162148
1,Kerala,Total,2819,1291516,1989,1783463,3754,830,0.529835,0.221097,0.750932,0.724162
4,Telangana,NON -PHARMA,51,40255,44,54113,61,7,0.721311,0.114754,0.836066,0.743906
5,Telangana,PHARMA,850,459594,808,542468,1022,42,0.790607,0.041096,0.831703,0.847228
2,Telangana,Total,901,499850,852,596582,1083,49,0.786704,0.045245,0.831948,0.837856
6,Total,,3738,1894378,2859,2483584,4854,879,0.588999,0.181088,0.770087,0.762760


In [353]:
state_summary=state_summary.loc[:,["State",'CTG','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [354]:
state_summary.head()

,State,CTG,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
0,Karnataka,PHARMA,20,18,0.900000,0,0.000000,18,0.900000,103538,103011,0.994910
0,Karnataka,Total,20,18,0.900000,0,0.000000,18,0.900000,103538,103011,0.994910
1,Kerala,NON -PHARMA,254,138,0.543307,37,0.145669,175,0.688976,149833,107768,0.719254
2,Kerala,PHARMA,3490,1851,0.530372,787,0.225501,2638,0.755874,1629793,1183125,0.725936
3,Kerala,PRIVATE_LABEL,10,0,0.000000,6,0.600000,6,0.600000,3836,622,0.162148


In [355]:
opsexe_summary=outside_purch.pivot_table(index=["State","City Group",'Ops Exexutive'],values=["Overall" ,"Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum",margins=True,margins_name="Total")
opsexe_summary.reset_index(inplace=True)

In [356]:
state_summary_st["Ops Exexutive"]="-"
state_summary_st["City Group"]="Total"

In [357]:
opsexe_summary = pd.concat([opsexe_summary, state_summary_st], sort=False).sort_values(['State', 'Ops Exexutive'])

In [358]:
opsexe_summary.reset_index(drop=True,inplace=True)

In [359]:
opsexe_summary

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG
0,Karnataka,Total,-,18.0,1.030119e+05,18.0,103538.96,20,0.0,Total
1,Karnataka,ROKA,Anilkumar M,4.0,1.959530e+03,4.0,2486.54,6,0.0,NaN
2,Karnataka,KA,Byregowda MR,13.0,2.531802e+04,13.0,25318.02,13,0.0,NaN
3,Karnataka,KA,Mohammed Ali A,1.0,7.573440e+04,1.0,75734.40,1,0.0,NaN
4,Kerala,Total,-,2819.0,1.291517e+06,1989.0,1783463.76,3754,830.0,Total
5,Kerala,KL,Akshai Chandran,11.0,3.724496e+03,10.0,6234.82,13,1.0,NaN
6,Kerala,ROKL,Asha Sk,520.0,1.628750e+05,391.0,203144.64,584,129.0,NaN
7,Kerala,KL,Boney Louies,33.0,9.751005e+03,29.0,17642.10,58,4.0,NaN
8,Kerala,KL,Fathima Rukzana,16.0,4.000457e+04,14.0,43615.71,23,2.0,NaN
9,Kerala,ROKL,Girish Kumar,1583.0,7.029150e+05,1050.0,1020042.44,2226,533.0,NaN


In [360]:
opsexe_summary_col_int = opsexe_summary.select_dtypes(include=['int', 'float']).columns
opsexe_summary[opsexe_summary_col_int]=opsexe_summary[opsexe_summary_col_int].astype(int)
opsexe_summary.head(3)

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG
0,Karnataka,Total,-,18,103011,18,103538,20,0,Total
1,Karnataka,ROKA,Anilkumar M,4,1959,4,2486,6,0,NaN
2,Karnataka,KA,Byregowda MR,13,25318,13,25318,13,0,NaN


In [361]:
opsexe_summary["FC %"]=opsexe_summary["Fully Conv"]/opsexe_summary["Overall"]
opsexe_summary["PC %"]=opsexe_summary["Partial Conv"]/opsexe_summary["Overall"]
opsexe_summary["Sku Con %"]=opsexe_summary["Converted"]/opsexe_summary["Overall"]
opsexe_summary["Sales Con %"]=opsexe_summary["Converted Value"]/opsexe_summary["OP Value"]

In [362]:
opsexe_summary.head()

,State,City Group,Ops Exexutive,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,CTG,FC %,PC %,Sku Con %,Sales Con %
0,Karnataka,Total,-,18,103011,18,103538,20,0,Total,0.900000,0.000000,0.900000,0.994910
1,Karnataka,ROKA,Anilkumar M,4,1959,4,2486,6,0,NaN,0.666667,0.000000,0.666667,0.788013
2,Karnataka,KA,Byregowda MR,13,25318,13,25318,13,0,NaN,1.000000,0.000000,1.000000,1.000000
3,Karnataka,KA,Mohammed Ali A,1,75734,1,75734,1,0,NaN,1.000000,0.000000,1.000000,1.000000
4,Kerala,Total,-,2819,1291516,1989,1783463,3754,830,Total,0.529835,0.221097,0.750932,0.724162


In [363]:
opsexe_summary=opsexe_summary.loc[:,["State",'City Group','Ops Exexutive','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [364]:
opsexe_summary.head()

,State,City Group,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
0,Karnataka,Total,-,20,18,0.900000,0,0.000000,18,0.900000,103538,103011,0.994910
1,Karnataka,ROKA,Anilkumar M,6,4,0.666667,0,0.000000,4,0.666667,2486,1959,0.788013
2,Karnataka,KA,Byregowda MR,13,13,1.000000,0,0.000000,13,1.000000,25318,25318,1.000000
3,Karnataka,KA,Mohammed Ali A,1,1,1.000000,0,0.000000,1,1.000000,75734,75734,1.000000
4,Kerala,Total,-,3754,1989,0.529835,830,0.221097,2819,0.750932,1783463,1291516,0.724162


In [365]:
store_summary=outside_purch.pivot_table(index=["State","Storeid","Outlet Name","Ops Exexutive","City Group"],values=["Overall","Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum")
store_summary.reset_index(inplace=True)
store_summary.head(3)

,State,Storeid,Outlet Name,Ops Exexutive,City Group,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,2901001,KA - Dasarahalli,Byregowda MR,KA,4.0,4286.69,4.0,4286.69,4,0.0
1,Karnataka,2901040,KA - Kamanahalli,Mohammed Ali A,KA,1.0,75734.40,1.0,75734.40,1,0.0
2,Karnataka,2901053,KA - Kirloskar Layout,Byregowda MR,KA,1.0,17135.81,1.0,17135.81,1,0.0


In [366]:
store_summary_col_int = store_summary.select_dtypes(include=['int', 'float']).columns
store_summary[store_summary_col_int]=store_summary[store_summary_col_int].astype(int)
store_summary.head(3)

,State,Storeid,Outlet Name,Ops Exexutive,City Group,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,2901001,KA - Dasarahalli,Byregowda MR,KA,4,4286,4,4286,4,0
1,Karnataka,2901040,KA - Kamanahalli,Mohammed Ali A,KA,1,75734,1,75734,1,0
2,Karnataka,2901053,KA - Kirloskar Layout,Byregowda MR,KA,1,17135,1,17135,1,0


In [367]:
store_summary["FC %"]=store_summary["Fully Conv"]/store_summary["Overall"]
store_summary["PC %"]=store_summary["Partial Conv"]/store_summary["Overall"]
store_summary["Sku Con %"]=store_summary["Converted"]/store_summary["Overall"]
store_summary["Sales Con %"]=store_summary["Converted Value"]/store_summary["OP Value"]

In [368]:
store_summary.head(1)

,State,Storeid,Outlet Name,Ops Exexutive,City Group,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv,FC %,PC %,Sku Con %,Sales Con %
0,Karnataka,2901001,KA - Dasarahalli,Byregowda MR,KA,4,4286,4,4286,4,0,1.0,0.0,1.0,1.0


In [369]:
store_summary=store_summary.loc[:,["State",'City Group','Storeid','Outlet Name','Ops Exexutive','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [370]:
store_summary.head(1)

,State,City Group,Storeid,Outlet Name,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
0,Karnataka,KA,2901001,KA - Dasarahalli,Byregowda MR,4,4,1.0,0,0.0,4,1.0,4286,4286,1.0


In [371]:
#pi_state_wise=pi_state_wise.sort_values(by=["State","YY-M","GRN Pendings",], ascending=[True,True,False])
#state_summary=state_summary.sort_values(by=["State","Sku Con %",], ascending=[True,False])
opsexe_summary=opsexe_summary.sort_values(by=["State","City Group","Sku Con %"], ascending=[True,True,True])
opsexe_sum7d=opsexe_sum7d.sort_values(by=["State","City Group","Sku Con %"], ascending=[True,True,True])
#pi_store_wise=pi_store_wise.sort_values(by=["State","GRN Pendings",], ascending=[True,False])

###### Exporting Setting

unique Settings

In [372]:
date=time.strftime("%d-%m-%Y")
date

'26-10-2023'

In [373]:
min_date=outside_purch["Purchase Date"].min()
print(min_date)
min_month=min_date.strftime("%d-%m-%Y")
min_month

2023-09-24 00:00:00


'24-09-2023'

In [374]:
max_date=outside_purch["Purchase Date"].max()
print(max_date)
max_month=max_date.strftime("%d-%m-%Y")
max_month

2023-10-24 00:00:00


'24-10-2023'

In [375]:
period="From "+min_month+" To "+max_month
period

'From 24-09-2023 To 24-10-2023'

####  Export Settings

In [376]:
filename="Outside Purchase Tracker "+period+" As on "+date

In [377]:
filename

'Outside Purchase Tracker From 24-09-2023 To 24-10-2023 As on 26-10-2023'

In [378]:
filename

'Outside Purchase Tracker From 24-09-2023 To 24-10-2023 As on 26-10-2023'

In [379]:
file_path_master="D:\Phyton raw data\OP VS Sales Conversion\\"+filename+".xlsx"

In [380]:
file_path_master

'D:\\Phyton raw data\\OP VS Sales Conversion\\Outside Purchase Tracker From 24-09-2023 To 24-10-2023 As on 26-10-2023.xlsx'

In [381]:
store_summary["Storeid"]=store_summary["Storeid"].astype(int)

#### Pan India Exports

In [382]:
writer = pd.ExcelWriter(file_path_master, engine='xlsxwriter')
state_summary.to_excel(writer, sheet_name='Summary', startrow=1, startcol=0,header=False, index=False)
opsexe_summary.to_excel(writer, sheet_name='Summary', startrow=1, startcol=13,header=False, index=False)
opsexe_sum7d.to_excel(writer, sheet_name='Last 07d Summary', startrow=1, startcol=0,header=False, index=False)
store_summary.to_excel(writer, sheet_name='Store Wise', startrow=1, startcol=0,header=False, index=False)
outside_purch.to_excel(writer, sheet_name='Raw Data', startrow=1, startcol=0,header=False, index=False)


workbook  = writer.book 
state_summary_s=writer.sheets['Summary']
opsexe_summary_s=writer.sheets['Summary']
opsexe_sum7d_s=writer.sheets['Last 07d Summary']
store_summary_s=writer.sheets['Store Wise']
outside_purch_s=writer.sheets['Raw Data']

header_format = workbook.add_format({'bold': True,'text_wrap': False,'valign': 'top','fg_color': '#034A90','font_color': 'white','border': 1})
header_format_2 = workbook.add_format({'bold': True,'text_wrap': False,'valign': 'top','fg_color': '#00b38c','font_color': 'white','border': 1})
header_format_3 = workbook.add_format({'bold': True,'text_wrap': False,'valign': 'top','fg_color': '#b2222','font_color': 'white','border': 1})

for col_num, value in enumerate(state_summary.columns.values):
    state_summary_s.write(0, col_num , value, header_format) 
    
for col_num, value in enumerate(opsexe_summary.columns.values):
    opsexe_summary_s.write(0, col_num+13 , value, header_format)
    
for col_num, value in enumerate(opsexe_sum7d.columns.values):
    opsexe_sum7d_s.write(0, col_num+0 , value, header_format_3)
    
for col_num, value in enumerate(store_summary.columns.values):
    store_summary_s.write(0, col_num , value, header_format_2)
    
for col_num, value in enumerate(outside_purch.columns.values):
    outside_purch_s.write(0, col_num , value, header_format_3)


    
writer.close()

###### Mail Setting Store wise

In [383]:
uniq_store=store_summary.loc[:,["Storeid","Outlet Name","State"]]

In [384]:
uniq_store["Storeid"]=uniq_store["Storeid"].astype(str)

In [385]:
uniq_store.duplicated().sum()

0

In [386]:
uniq_store.reset_index(drop=True ,inplace=True)

In [387]:
uniq_store["Period"]="Last 30 days"
uniq_store["Period2"]="Last 07 days"
uniq_store.tail()

,Storeid,Outlet Name,State,Period,Period2
91,3601055,TG - Balapur X Road,Telangana,Last 30 days,Last 07 days
92,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana,Last 30 days,Last 07 days
93,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,Last 30 days,Last 07 days
94,3601059,TG - Film Nagar,Telangana,Last 30 days,Last 07 days
95,3601062,TG - Moinabad,Telangana,Last 30 days,Last 07 days


In [388]:
uniq_store=pd.merge(uniq_store,sd_details[["Storeid",'Email ID']],on="Storeid",how='left')

In [389]:
uniq_store["Storeid"]=uniq_store["Storeid"].astype(int)

In [390]:
period_l30=uniq_store.loc[:,['Period','Storeid','Outlet Name',"State"]]

In [391]:
period_l7=uniq_store.loc[:,['Period2','Storeid','Outlet Name',"State"]]

In [392]:
period_l30.tail()

,Period,Storeid,Outlet Name,State
91,Last 30 days,3601055,TG - Balapur X Road,Telangana
92,Last 30 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana
93,Last 30 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana
94,Last 30 days,3601059,TG - Film Nagar,Telangana
95,Last 30 days,3601062,TG - Moinabad,Telangana


In [393]:
period_l7.rename(columns={"Period2":"Period"},inplace=True)
period_l7.tail()

,Period,Storeid,Outlet Name,State
91,Last 07 days,3601055,TG - Balapur X Road,Telangana
92,Last 07 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana
93,Last 07 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana
94,Last 07 days,3601059,TG - Film Nagar,Telangana
95,Last 07 days,3601062,TG - Moinabad,Telangana


In [394]:
store_summary.columns

Index(['State', 'City Group', 'Storeid', 'Outlet Name', 'Ops Exexutive',
       'Overall', 'Fully Conv', 'FC %', 'Partial Conv', 'PC %', 'Converted',
       'Sku Con %', 'OP Value', 'Converted Value', 'Sales Con %'],
      dtype='object')

In [395]:
period_l30=pd.merge(period_l30,store_summary[['Storeid','Overall', 'Fully Conv', 'FC %', 'Partial Conv', 'PC %', 'Converted',
       'Sku Con %', 'OP Value', 'Converted Value', 'Sales Con %']],on="Storeid",how='left')

In [396]:
period_l30.tail()

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
91,Last 30 days,3601055,TG - Balapur X Road,Telangana,50,40,0.800000,2,0.040000,42,0.840000,24168,19961,0.825927
92,Last 30 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana,8,8,1.000000,0,0.000000,8,1.000000,7594,7594,1.000000
93,Last 30 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,89,76,0.853933,0,0.000000,76,0.853933,37485,34663,0.924717
94,Last 30 days,3601059,TG - Film Nagar,Telangana,3,1,0.333333,1,0.333333,2,0.666667,3469,905,0.260882
95,Last 30 days,3601062,TG - Moinabad,Telangana,28,23,0.821429,2,0.071429,25,0.892857,9499,8585,0.903779


In [397]:
store_summary_l7=outside_purch[outside_purch["Days"]<="7 Days"].pivot_table(index=["State","Storeid","Outlet Name","Ops Exexutive","City Group"],values=["Overall","Fully Conv","Partial Conv","Converted","OP Value","Converted Value"],aggfunc="sum")
store_summary_l7.reset_index(inplace=True)
print(store_summary_l7.shape)
store_summary_l7.head(3)

(52, 11)


,State,Storeid,Outlet Name,Ops Exexutive,City Group,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,2901001,KA - Dasarahalli,Byregowda MR,KA,1.0,985.600000,1.0,985.60,1,0.0
1,Karnataka,2901059,KA - Kodigehalli Main Road,Byregowda MR,KA,1.0,1290.240000,1.0,1290.24,1,0.0
2,Kerala,3201005,KL - Aluva,Fathima Rukzana,KL,5.0,901.428333,4.0,1962.05,8,1.0


In [398]:
store_summary_l7_col_int = store_summary_l7.select_dtypes(include=['int', 'float']).columns
store_summary_l7[store_summary_l7_col_int]=store_summary_l7[store_summary_l7_col_int].astype(int)
print(store_summary_l7.shape)
store_summary_l7.head(3)

(52, 11)


,State,Storeid,Outlet Name,Ops Exexutive,City Group,Converted,Converted Value,Fully Conv,OP Value,Overall,Partial Conv
0,Karnataka,2901001,KA - Dasarahalli,Byregowda MR,KA,1,985,1,985,1,0
1,Karnataka,2901059,KA - Kodigehalli Main Road,Byregowda MR,KA,1,1290,1,1290,1,0
2,Kerala,3201005,KL - Aluva,Fathima Rukzana,KL,5,901,4,1962,8,1


In [399]:
store_summary_l7["FC %"]=store_summary_l7["Fully Conv"]/store_summary_l7["Overall"]
store_summary_l7["PC %"]=store_summary_l7["Partial Conv"]/store_summary_l7["Overall"]
store_summary_l7["Sku Con %"]=store_summary_l7["Converted"]/store_summary_l7["Overall"]
store_summary_l7["Sales Con %"]=store_summary_l7["Converted Value"]/store_summary_l7["OP Value"]

In [400]:
store_summary_l7=store_summary_l7.loc[:,["State",'City Group','Storeid','Outlet Name','Ops Exexutive','Overall',"Fully Conv","FC %","Partial Conv","PC %",'Converted',"Sku Con %",'OP Value','Converted Value','Sales Con %']]

In [401]:
store_summary_l7.tail(2)

,State,City Group,Storeid,Outlet Name,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
50,Telangana,TG,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Renatla Surender,25,19,0.76,0,0.0,19,0.76,10884,10151,0.932653
51,Telangana,ROTG,3601062,TG - Moinabad,Renatla Surender,2,2,1.00,0,0.0,2,1.00,534,534,1.000000


In [402]:
store_summary_l7["Storeid"]=store_summary_l7["Storeid"].astype(int)
store_summary_l7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   State            52 non-null     object 
 1   City Group       52 non-null     object 
 2   Storeid          52 non-null     int32  
 3   Outlet Name      52 non-null     object 
 4   Ops Exexutive    52 non-null     object 
 5   Overall          52 non-null     int32  
 6   Fully Conv       52 non-null     int32  
 7   FC %             52 non-null     float64
 8   Partial Conv     52 non-null     int32  
 9   PC %             52 non-null     float64
 10  Converted        52 non-null     int32  
 11  Sku Con %        52 non-null     float64
 12  OP Value         52 non-null     int32  
 13  Converted Value  52 non-null     int32  
 14  Sales Con %      52 non-null     float64
dtypes: float64(4), int32(7), object(4)
memory usage: 4.8+ KB


In [403]:
period_l7=pd.merge(period_l7,store_summary_l7[['Storeid','Overall', 'Fully Conv', 'FC %', 'Partial Conv', 'PC %', 'Converted',
       'Sku Con %', 'OP Value', 'Converted Value', 'Sales Con %']],on="Storeid",how='left')

In [404]:
period_l7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Period           96 non-null     object 
 1   Storeid          96 non-null     int32  
 2   Outlet Name      96 non-null     object 
 3   State            96 non-null     object 
 4   Overall          52 non-null     float64
 5   Fully Conv       52 non-null     float64
 6   FC %             52 non-null     float64
 7   Partial Conv     52 non-null     float64
 8   PC %             52 non-null     float64
 9   Converted        52 non-null     float64
 10  Sku Con %        52 non-null     float64
 11  OP Value         52 non-null     float64
 12  Converted Value  52 non-null     float64
 13  Sales Con %      52 non-null     float64
dtypes: float64(10), int32(1), object(3)
memory usage: 10.9+ KB


In [405]:
period_l7.fillna(0,inplace=True)
period_l7.tail()

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
91,Last 07 days,3601055,TG - Balapur X Road,Telangana,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000
92,Last 07 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000
93,Last 07 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,25.0,19.0,0.76,0.0,0.0,19.0,0.76,10884.0,10151.0,0.932653
94,Last 07 days,3601059,TG - Film Nagar,Telangana,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.000000
95,Last 07 days,3601062,TG - Moinabad,Telangana,2.0,2.0,1.00,0.0,0.0,2.0,1.00,534.0,534.0,1.000000


In [406]:
period_l30["Storeid"]=period_l30["Storeid"].astype(str)
period_l7["Storeid"]=period_l7["Storeid"].astype(str)
period_l30.tail()

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
91,Last 30 days,3601055,TG - Balapur X Road,Telangana,50,40,0.800000,2,0.040000,42,0.840000,24168,19961,0.825927
92,Last 30 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana,8,8,1.000000,0,0.000000,8,1.000000,7594,7594,1.000000
93,Last 30 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,89,76,0.853933,0,0.000000,76,0.853933,37485,34663,0.924717
94,Last 30 days,3601059,TG - Film Nagar,Telangana,3,1,0.333333,1,0.333333,2,0.666667,3469,905,0.260882
95,Last 30 days,3601062,TG - Moinabad,Telangana,28,23,0.821429,2,0.071429,25,0.892857,9499,8585,0.903779


In [407]:
period_l30=pd.merge(period_l30,sd_details[["Storeid",'Email ID','Cluster Mail Id']],on="Storeid",how='left')
period_l7=pd.merge(period_l7,sd_details[["Storeid",'Email ID','Cluster Mail Id']],on="Storeid",how='left')

In [408]:
period_l7["FC %"]=(period_l7["FC %"]*100).astype(int).astype(str)+"%"
period_l7["PC %"]=(period_l7["PC %"]*100).astype(int).astype(str)+"%"
period_l7["Sku Con %"]=(period_l7["Sku Con %"]*100).astype(int).astype(str)+"%"
period_l7["Sales Con %"] = (period_l7["Sales Con %"] * 100).fillna(0).astype(int)

In [409]:
period_l30["FC %"]=(period_l30["FC %"]*100).astype(int).astype(str)+"%"
period_l30["PC %"]=(period_l30["PC %"]*100).astype(int).astype(str)+"%"
period_l30["Sku Con %"]=(period_l30["Sku Con %"]*100).astype(int).astype(str)+"%"
period_l30["Sales Con %"] = (period_l30["Sales Con %"] * 100).fillna(0).astype(int)

In [410]:
stwsie_l30_l7=pd.concat([period_l30,period_l7])

In [411]:
"""stwsie_l30_l7["FC %"]=(stwsie_l30_l7["FC %"]*100).astype(int).astype(str)+"%"
stwsie_l30_l7["PC %"]=(stwsie_l30_l7["PC %"]*100).astype(int).astype(str)+"%"
stwsie_l30_l7["Sku Con %"]=(stwsie_l30_l7["Sku Con %"]*100).astype(int).astype(str)+"%"
stwsie_l30_l7["Sales Con %"] = (stwsie_l30_l7["Sales Con %"] * 100).fillna(0).astype(int)"""

'stwsie_l30_l7["FC %"]=(stwsie_l30_l7["FC %"]*100).astype(int).astype(str)+"%"\nstwsie_l30_l7["PC %"]=(stwsie_l30_l7["PC %"]*100).astype(int).astype(str)+"%"\nstwsie_l30_l7["Sku Con %"]=(stwsie_l30_l7["Sku Con %"]*100).astype(int).astype(str)+"%"\nstwsie_l30_l7["Sales Con %"] = (stwsie_l30_l7["Sales Con %"] * 100).fillna(0).astype(int)'

In [412]:
stwsie_l30_l7.tail(5)

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %,Email ID,Cluster Mail Id
91,Last 07 days,3601055,TG - Balapur X Road,Telangana,0.0,0.0,0%,0.0,0%,0.0,0%,0.0,0.0,0,36010055@asterpharmacy.in,kappe.ramesh@asterpharmacy.in
92,Last 07 days,3601057,TG - Sri Balaji Layout Gajularamaram,Telangana,0.0,0.0,0%,0.0,0%,0.0,0%,0.0,0.0,0,36010057@asterpharmacy.in,sikander.r@asterpharmacy.in
93,Last 07 days,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,25.0,19.0,76%,0.0,0%,19.0,76%,10884.0,10151.0,93,36010058@asterpharmacy.in,renatla.surendar@asterpharmacy.in
94,Last 07 days,3601059,TG - Film Nagar,Telangana,0.0,0.0,0%,0.0,0%,0.0,0%,0.0,0.0,0,36010059@asterpharmacy.in,renatla.surendar@asterpharmacy.in
95,Last 07 days,3601062,TG - Moinabad,Telangana,2.0,2.0,100%,0.0,0%,2.0,100%,534.0,534.0,100,36010062@asterpharmacy.in,renatla.surendar@asterpharmacy.in


In [413]:
sd_details.columns

Index(['S. No.', 'Store Code', 'Store Type', 'HDH Store Code', 'HPD BU2 ID',
       'HPD TNG', 'Get Well', 'SreeJi', 'Vasu agency', 'Venkat Sai',
       'HPD Kerala', 'PO Store Name', 'Store NameAs per Branding ',
       'Outlet Name', 'GOF Store Id', 'Launch Date', 'Email ID', 'Short Name',
       'Build-up Area in Sft', 'Phone Number', 'Ops Exexutive',
       'Cluster Mail Id', 'Executive Phone Number', 'Area Manager',
       'Sub Region', 'State', 'City', 'City 2', 'Gof_status', 'Manager',
       'Cluster', 'Any Desk ID', 'Rental', 'FF Target', 'Address', 'Zip Code',
       'Region', 'HOO', 'Outlet Format', 'Tenure Days', 'Tenure Month',
       'Tenure Years', 'Tenure', 'Launch Month', 'Launch Year', 'Quarter',
       'Financial Year', 'Securty Deposit', 'Rent Start Date', 'Store System',
       'Storeid', 'Store Name As per D 365', 'Retail Channel Id', 'Party ID',
       'Fin And Qtr', 'Location Series', 'City G', 'Vendor Code'],
      dtype='object')

In [414]:
stwsie_l30_l7.head(3)

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %,Email ID,Cluster Mail Id
0,Last 30 days,2901001,KA - Dasarahalli,Karnataka,4.0,4.0,100%,0.0,0%,4.0,100%,4286.0,4286.0,100,29010001@asterpharmacy.in,byregowda.mr@asterpharmacy.in
1,Last 30 days,2901040,KA - Kamanahalli,Karnataka,1.0,1.0,100%,0.0,0%,1.0,100%,75734.0,75734.0,100,29010040@asterpharmacy.in,mohammed.a@asterpharmacy.in
2,Last 30 days,2901053,KA - Kirloskar Layout,Karnataka,1.0,1.0,100%,0.0,0%,1.0,100%,17135.0,17135.0,100,29010053@asterpharmacy.in,byregowda.mr@asterpharmacy.in


In [415]:
#stwsie_l30_l7["Store_mailid"]=stwsie_l30_l7["Email ID"]

In [416]:
#stwsie_l30_l7.drop("Email ID",axis=1,inplace=True)

In [417]:
stwsie_l30_l7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 95
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Period           192 non-null    object 
 1   Storeid          192 non-null    object 
 2   Outlet Name      192 non-null    object 
 3   State            192 non-null    object 
 4   Overall          192 non-null    float64
 5   Fully Conv       192 non-null    float64
 6   FC %             192 non-null    object 
 7   Partial Conv     192 non-null    float64
 8   PC %             192 non-null    object 
 9   Converted        192 non-null    float64
 10  Sku Con %        192 non-null    object 
 11  OP Value         192 non-null    float64
 12  Converted Value  192 non-null    float64
 13  Sales Con %      192 non-null    int32  
 14  Email ID         192 non-null    object 
 15  Cluster Mail Id  192 non-null    object 
dtypes: float64(6), int32(1), object(9)
memory usage: 24.8+ KB


###### Attachment Save by Store wise

In [418]:
folder_name=tday.strftime('%d-%m-%Y')

In [419]:
folder_name

'26-10-2023'

###### Store Wise Splitting

In [420]:
outside_purch.columns

Index(['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG', 'Pur Qty In strips', 'OP Value',
       'Outlet Name', 'State', 'City Group', 'Ops Exexutive', 'Cluster',
       'Store Type', 'start_date', 'Days', 'One MDM Code', 'Unit', 'T-date',
       'Tra Store id', 'Total Trasanferd qty', 'Overall',
       'Xfer(D) & Recv(D)in Tostore', 'RMNG PNDG in Fromstore', 'Purchase Qty',
       'After Transfer sold Qty', 'After Purchase sold Qty', 'Total Sold Qty',
       'Sold Qty', 'Converted Value', 'Sales Status', 'Conv on Recv(D)store',
       'Sales Biferication', 'Converted', 'Fully Conv', 'Partial Conv'],
      dtype='object')

In [421]:
raw_data=outside_purch.loc[:,['Purchase Date', 'Storeid', 'StoreName', 'VendName', 'Item Code',
       'ItemName', 'Vendor Group', 'CTG', 'Pur Qty In strips', 'OP Value',
       'Outlet Name', 'State', 'Sold Qty', 'Converted Value', 'Sales Biferication']]

In [422]:
raw_data.tail(2)

,Purchase Date,Storeid,StoreName,VendName,Item Code,ItemName,Vendor Group,CTG,Pur Qty In strips,OP Value,Outlet Name,State,Sold Qty,Converted Value,Sales Biferication
4855,2023-10-24,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,PRIYANKA PHARMA,920816,RENODAPT S 360 TABLET,Outside Purchase,PHARMA,1.0,852.00,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,10.0,852.0,Converted
4856,2023-10-24,3601058,TG - Ayodhya Nagar Colony Mehdipatnam,PRIYANKA PHARMA,928004,SCALPE PRO 100ML SHAMPOO,Outside Purchase,PHARMA,1.0,165.74,TG - Ayodhya Nagar Colony Mehdipatnam,Telangana,0.0,0.0,Not Converted


In [423]:
folder_name_path = 'D:\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\Aster Pharmacy Raw data\Phyton RD Drive\Store Wise SPlitted\Purchase Register'
if not os.path.exists(folder_name_path):    
    os.makedirs(folder_name_path)

In [424]:
folder_name_path

'D:\\OneDrive - ALFAONE RETAIL PHARMACIES PRIVATE LIMITED\\Aster Pharmacy Raw data\\Phyton RD Drive\\Store Wise SPlitted\\Purchase Register'

###### Unique Values

In [425]:
unique_values=raw_data['Storeid'].value_counts().count()

In [426]:
unique_values

96

In [427]:
raw_data_val=raw_data['Storeid'].unique()

In [428]:
for items in raw_data_val:
    df = raw_data[raw_data["Storeid"]==items]
    path = os.path.join(folder_name_path, str(items)+".xlsx")
    #df.set_index('Po FIleName', inplace=True)
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    for store_id in df['Storeid'].unique():
        df_temp = df[df['Storeid']==store_id]
        sheet_name = str(store_id)
        
        df_temp.to_excel(writer, sheet_name=sheet_name, index=False)
    writer.save()

##### Store wise Mail Allocation with Attachment

In [429]:
smtp = smtplib.SMTP('smtp.office365.com', 587)
smtp.ehlo()
smtp.starttls()

smtp.login('inventory.ho@asterpharmacy.in', 'Aster@126277')

(235, b'2.7.0 Authentication successful')

In [430]:
stwsie_l30_l7.tail(2)

,Period,Storeid,Outlet Name,State,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %,Email ID,Cluster Mail Id
94,Last 07 days,3601059,TG - Film Nagar,Telangana,0.0,0.0,0%,0.0,0%,0.0,0%,0.0,0.0,0,36010059@asterpharmacy.in,renatla.surendar@asterpharmacy.in
95,Last 07 days,3601062,TG - Moinabad,Telangana,2.0,2.0,100%,0.0,0%,2.0,100%,534.0,534.0,100,36010062@asterpharmacy.in,renatla.surendar@asterpharmacy.in


In [431]:
regards="<p style='font-family: tahoma;'><font color=darkblue><b>From</b> <br>Team Inventory- HQ<br><b>Aster Pharmacy ,</b><br>Renaissance Centra, 27 & 27/1,  Ward 77, <br>Mission Road, Sampangi Rama Nagar, Bengaluru, Karnataka 560027, India .<br></font ></p>"

In [432]:
HTML(regards)

In [433]:
#Code to send sales table to respective store emails (live)
ftd1 = tday.strftime('%d-%m-%y')
#ftd1 = ftd
i = 0

for ind in stwsie_l30_l7.index:
    if ind == i:
        output = stwsie_l30_l7.loc[ind,:]
        output = output.drop(['Email ID','Cluster Mail Id'],axis=1)
        output = build_table(output, 'blue_dark',font_size='10px',font_family='tahoma', width='100px',text_align = 'center')
        msg = MIMEMultipart()
        msg['Subject'] = "Outside Purchase Vs Sales Conversation ||"+period
        msg.attach(MIMEText(f'<p>Dear Store Team,<br><br>Please find below the Outside Purchase Vs Sales Conversation {period}.<br>{output}<br>{regards}',
                    'html'))
        to = [stwsie_l30_l7.iloc[ind,14]]
        cc = [stwsie_l30_l7.iloc[ind,15]]
        msg['To'] = ', '.join(to)
        msg['Cc'] = ', '.join(cc)
        
        store_id = stwsie_l30_l7.iloc[ind, 1]  # Replace YOUR_STORE_ID_COLUMN_INDEX with the correct index
        file_path = os.path.join(folder_name_path, f"{store_id}.xlsx")  # Adjust the file extension as needed

        with open(file_path, 'rb') as file:
            attachment = MIMEApplication(file.read(), _subtype="xlsx")
            attachment.add_header('Content-Disposition', 'attachment', filename=f"{store_id}.xlsx")
            msg.attach(attachment)
        
        msg_text = msg.as_string()
        smtp.sendmail(from_addr="inventory.ho@asterpharmacy.in", to_addrs=to + cc, msg=msg_text)
        i = i+1
        
    
        
#smtp.quit()

#### HTML Convertion Mail

In [434]:
state_sum7d["FC %"]=round(state_sum7d["FC %"]*100,2)
state_sum7d["PC %"]=round(state_sum7d["PC %"]*100,2)
state_sum7d["Sku Con %"]=round(state_sum7d["Sku Con %"]*100,2)
state_sum7d["Sales Con %"]=round(state_sum7d["Sales Con %"]*100,2)

In [435]:
opsexe_sum7d["FC %"]=round(opsexe_sum7d["FC %"]*100,2)
opsexe_sum7d["PC %"]=round(opsexe_sum7d["PC %"]*100,2)
opsexe_sum7d["Sku Con %"]=round(opsexe_sum7d["Sku Con %"]*100,2)
opsexe_sum7d["Sales Con %"]=round(opsexe_sum7d["Sales Con %"]*100,2)

In [436]:
state_summary["FC %"]=round(state_summary["FC %"]*100,2)
state_summary["PC %"]=round(state_summary["PC %"]*100,2)
state_summary["Sku Con %"]=round(state_summary["Sku Con %"]*100,2)
state_summary["Sales Con %"]=round(state_summary["Sales Con %"]*100,2)

In [437]:
opsexe_summary["FC %"]=round(opsexe_summary["FC %"]*100,2)
opsexe_summary["PC %"]=round(opsexe_summary["PC %"]*100,2)
opsexe_summary["Sku Con %"]=round(opsexe_summary["Sku Con %"]*100,2)
opsexe_summary["Sales Con %"]=round(opsexe_summary["Sales Con %"]*100,2)

In [438]:
opsexe_summary.head(2)

,State,City Group,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
2,Karnataka,KA,Byregowda MR,13,13,100.0,0,0.0,13,100.0,25318,25318,100.0
3,Karnataka,KA,Mohammed Ali A,1,1,100.0,0,0.0,1,100.0,75734,75734,100.0


In [439]:
state_sum7d.reset_index(inplace=True)
state_sum7d.drop("index",axis=1,inplace=True)

In [440]:
def format_completed_percentage(value):
    return 'background-color: #ff8c69; color: black' if value < 80.0 else 'background-color: #ffd700' if value < 90.99 else 'background-color: #9acd32; color: black'

def format_pending_percentage(value):
    return 'background-color: #ff8c69; color: black' if value > 1.0 else 'background-color: #ffd700' if value > 0.01 else 'background-color: #9acd32; color: black'


def highlight_total_row(row):
    if row['CTG'] == 'Total':
        return ['background-color: #00ac8a; color: white; font-weight: bold'] * len(row)
    return [''] * len(row)

def highlight_total_row_1(row):
    if row['City Group'] == '-':
        return ['background-color: #00ac8a; color: white; font-weight: bold'] * len(row)
    return [''] * len(row)

def highlight_total_row_2(row):
    if row['State'] == 'Total':
        return ['background-color: #00ac8a; color: white; font-weight: bold'] * len(row)
    return [''] * len(row)

def highlight_total_row_3(row):
    if row['City Group'] == 'Total':
        return ['background-color: #00ac8a; color: white; font-weight: bold'] * len(row)
    return [''] * len(row)

In [441]:
styled_state_sum7d = state_sum7d.style \
    .format({'Sku Con %': '{:.2f}%', 'Sales Con %': '{:.2f}%', 'FC %': '{:.2f}%', 'PC %': '{:.2f}%'}).hide(axis='index') \
    .applymap(format_completed_percentage, subset='Sku Con %') \
    .applymap(format_completed_percentage, subset='Sales Con %') \
    .apply(highlight_total_row_2, axis=1) \
    .apply(highlight_total_row, axis=1)

styled_state_sum7d.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#005596'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center'), ('vertical-align', 'middle')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('vertical-align', 'middle')]}
])

state_sum7d_html = styled_state_sum7d.to_html()
state_sum7d_html = state_sum7d_html.replace('<table', '<table style="border-collapse: collapse; border: 1px solid black;"')
state_sum7d_html = state_sum7d_html.replace('<td', '<td style="border: 1px solid black;"')

In [442]:
HTML(state_sum7d_html)

State,CTG,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
Karnataka,PHARMA,2,2,100.00%,0,0.00%,2,100.00%,2275,2275,100.00%
Karnataka,Total,2,2,100.00%,0,0.00%,2,100.00%,2275,2275,100.00%
Kerala,NON -PHARMA,28,13,46.43%,7,25.00%,20,71.43%,13688,11870,86.72%
Kerala,PHARMA,710,306,43.10%,184,25.92%,490,69.01%,310289,196209,63.23%
Kerala,Total,738,319,43.22%,191,25.88%,510,69.11%,323977,208080,64.23%
Telangana,NON -PHARMA,6,2,33.33%,1,16.67%,3,50.00%,5338,2935,54.98%
Telangana,PHARMA,141,99,70.21%,5,3.55%,104,73.76%,94293,76715,81.36%
Telangana,Total,147,101,68.71%,6,4.08%,107,72.79%,99632,79650,79.94%
Total,,886,422,47.63%,197,22.23%,619,69.86%,425885,290007,68.10%


In [443]:
opsexe_sum7d.reset_index(inplace=True)
opsexe_sum7d.drop("index",axis=1,inplace=True)

In [444]:
styled_opsexe_sum7d = opsexe_sum7d.style \
    .format({'Sku Con %': '{:.2f}%', 'Sales Con %': '{:.2f}%', 'FC %': '{:.2f}%', 'PC %': '{:.2f}%'}).hide(axis='index') \
    .applymap(format_completed_percentage, subset='Sku Con %') \
    .applymap(format_completed_percentage, subset='Sales Con %') \
    .apply(highlight_total_row_2, axis=1) \
    .apply(highlight_total_row_1, axis=1)

styled_opsexe_sum7d.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#005596'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center'), ('vertical-align', 'middle')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('vertical-align', 'middle')]}
])

opsexe_sum7d_html = styled_opsexe_sum7d.to_html()
opsexe_sum7d_html = opsexe_sum7d_html.replace('<table', '<table style="border-collapse: collapse; border: 1px solid black;"')
opsexe_sum7d_html = opsexe_sum7d_html.replace('<td', '<td style="border: 1px solid black;"')

In [445]:
HTML(opsexe_sum7d_html)

State,City Group,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
Karnataka,-,Total,2,2,100.00%,0,0.00%,2,100.00%,2275,2275,100.00%
Karnataka,KA,Byregowda MR,2,2,100.00%,0,0.00%,2,100.00%,2275,2275,100.00%
Kerala,-,Total,738,319,43.22%,191,25.88%,510,69.11%,323977,208080,64.23%
Kerala,KL,Fathima Rukzana,11,7,63.64%,1,9.09%,8,72.73%,13138,12077,91.92%
Kerala,KL,Akshai Chandran,5,3,60.00%,1,20.00%,4,80.00%,2032,794,39.07%
Kerala,ROKL,Girish Kumar,402,135,33.58%,104,25.87%,239,59.45%,142811,76595,53.63%
Kerala,ROKL,Prejith Mt,163,86,52.76%,29,17.79%,115,70.55%,100712,64972,64.51%
Kerala,ROKL,Asha Sk,154,85,55.19%,56,36.36%,141,91.56%,55701,44058,79.10%
Kerala,ROKL,Jithin M,3,3,100.00%,0,0.00%,3,100.00%,9581,9581,100.00%
Telangana,-,Total,147,101,68.71%,6,4.08%,107,72.79%,99632,79650,79.94%


In [446]:
state_summary.reset_index(inplace=True)
state_summary.drop("index",axis=1,inplace=True)
opsexe_summary.reset_index(inplace=True)
opsexe_summary.drop("index",axis=1,inplace=True)

In [447]:
styled_state_summary = state_summary.style \
    .format({'Sku Con %': '{:.2f}%', 'Sales Con %': '{:.2f}%', 'FC %': '{:.2f}%', 'PC %': '{:.2f}%'}).hide(axis='index') \
    .applymap(format_completed_percentage, subset='Sku Con %') \
    .applymap(format_completed_percentage, subset='Sales Con %') \
    .apply(highlight_total_row_2, axis=1) \
    .apply(highlight_total_row, axis=1)

styled_state_summary.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#005596'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center'), ('vertical-align', 'middle')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('vertical-align', 'middle')]}
])

state_summary_html = styled_state_summary.to_html()
state_summary_html = state_summary_html.replace('<table', '<table style="border-collapse: collapse; border: 1px solid black;"')
state_summary_html = state_summary_html.replace('<td', '<td style="border: 1px solid black;"')

In [448]:
HTML(state_summary_html)

State,CTG,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
Karnataka,PHARMA,20,18,90.00%,0,0.00%,18,90.00%,103538,103011,99.49%
Karnataka,Total,20,18,90.00%,0,0.00%,18,90.00%,103538,103011,99.49%
Kerala,NON -PHARMA,254,138,54.33%,37,14.57%,175,68.90%,149833,107768,71.93%
Kerala,PHARMA,3490,1851,53.04%,787,22.55%,2638,75.59%,1629793,1183125,72.59%
Kerala,PRIVATE_LABEL,10,0,0.00%,6,60.00%,6,60.00%,3836,622,16.21%
Kerala,Total,3754,1989,52.98%,830,22.11%,2819,75.09%,1783463,1291516,72.42%
Telangana,NON -PHARMA,61,44,72.13%,7,11.48%,51,83.61%,54113,40255,74.39%
Telangana,PHARMA,1022,808,79.06%,42,4.11%,850,83.17%,542468,459594,84.72%
Telangana,Total,1083,852,78.67%,49,4.52%,901,83.19%,596582,499850,83.79%
Total,,4854,2859,58.90%,879,18.11%,3738,77.01%,2483584,1894378,76.28%


In [449]:
styled_opsexe_summary = opsexe_summary.style \
    .format({'Sku Con %': '{:.2f}%', 'Sales Con %': '{:.2f}%', 'FC %': '{:.2f}%', 'PC %': '{:.2f}%'}).hide(axis='index') \
    .applymap(format_completed_percentage, subset='Sku Con %') \
    .applymap(format_completed_percentage, subset='Sales Con %') \
    .apply(highlight_total_row_2, axis=1) \
    .apply(highlight_total_row_3, axis=1)

styled_opsexe_summary.set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#005596'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'center'), ('vertical-align', 'middle')]},
    {'selector': 'td', 'props': [('text-align', 'center'), ('vertical-align', 'middle')]}
])

opsexe_summary_html = styled_opsexe_summary.to_html()
opsexe_summary_html = opsexe_summary_html.replace('<table', '<table style="border-collapse: collapse; border: 1px solid black;"')
opsexe_summary_html = opsexe_summary_html.replace('<td', '<td style="border: 1px solid black;"')

In [450]:
HTML(opsexe_summary_html)

State,City Group,Ops Exexutive,Overall,Fully Conv,FC %,Partial Conv,PC %,Converted,Sku Con %,OP Value,Converted Value,Sales Con %
Karnataka,KA,Byregowda MR,13,13,100.00%,0,0.00%,13,100.00%,25318,25318,100.00%
Karnataka,KA,Mohammed Ali A,1,1,100.00%,0,0.00%,1,100.00%,75734,75734,100.00%
Karnataka,ROKA,Anilkumar M,6,4,66.67%,0,0.00%,4,66.67%,2486,1959,78.80%
Karnataka,Total,-,20,18,90.00%,0,0.00%,18,90.00%,103538,103011,99.49%
Kerala,KL,Nikhil Antony,109,10,9.17%,20,18.35%,30,27.52%,31116,6007,19.31%
Kerala,KL,Boney Louies,58,29,50.00%,4,6.90%,33,56.90%,17642,9751,55.27%
Kerala,KL,Rahul Rc,69,26,37.68%,20,28.99%,46,66.67%,34054,14074,41.33%
Kerala,KL,Fathima Rukzana,23,14,60.87%,2,8.70%,16,69.57%,43615,40004,91.72%
Kerala,KL,Akshai Chandran,13,10,76.92%,1,7.69%,11,84.62%,6234,3724,59.74%
Kerala,ROKL,Girish Kumar,2226,1050,47.17%,533,23.94%,1583,71.11%,1020042,702915,68.91%


In [451]:
opvssales_sub="||Outside Purchase Vs Sales Conversion |"+period+"||"

In [452]:
opvssales_sub

'||Outside Purchase Vs Sales Conversion |From 24-09-2023 To 24-10-2023||'

In [453]:
htmlbody='<p>Dear Team,<BR><br>Please find attached '+filename+'<br><p> <u><b>Last 7 Days :-</b> </u><br>'+state_sum7d_html+'<br>'+opsexe_sum7d_html+'<br><u><b>Last 30 Days :-</b> </u><br>'+state_summary_html+'<br>'+opsexe_summary_html+'<br><br><p> <p><br>'+regards+'<p>'

In [454]:
HTML(htmlbody)

###### Email Settings

In [455]:
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

In [456]:
"""mail = outlook.CreateItem(0)
#mail.To = 'Jagadeesh.k@asterpharmacy.in'
mail.To = 'karnatakaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com;Keralaops.asterpharmcy@alfaonepharmacy.onmicrosoft.com;Telanganaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com'
mail.cc = 'inventory.asterpharmacy@alfaonepharmacy.onmicrosoft.com;mohammed.arfath@asterpharmacy.in;guruvayur.maharana@asterpharmacy.in;harsha.g@asterpharmacy.in;mudassar.mushtaq@asterpharmacy.in;madhu.n@hindustanpharma.co.in;rajesh.d@asterpharmacy.in;rahul.m@asterpharmacy.in;tom.saji@asterpharmacy.in'
mail.Subject =opvssales_sub
#mail.Body = 'Hi Team '
mail.HTMLBody = htmlbody #this field is optional

# To attach a file to the email (optional):
attachment1  = file_path_master
#attachment2  = kl_purch_lines
#attachment3  = tng_purch_lines

mail.Attachments.Add(attachment1)
#mail.Attachments.Add(attachment2)
#mail.Attachments.Add(attachment3)

mail.Send()
"""

"mail = outlook.CreateItem(0)\n#mail.To = 'Jagadeesh.k@asterpharmacy.in'\nmail.To = 'karnatakaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com;Keralaops.asterpharmcy@alfaonepharmacy.onmicrosoft.com;Telanganaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com'\nmail.cc = 'inventory.asterpharmacy@alfaonepharmacy.onmicrosoft.com;mohammed.arfath@asterpharmacy.in;guruvayur.maharana@asterpharmacy.in;harsha.g@asterpharmacy.in;mudassar.mushtaq@asterpharmacy.in;madhu.n@hindustanpharma.co.in;rajesh.d@asterpharmacy.in;rahul.m@asterpharmacy.in;tom.saji@asterpharmacy.in'\nmail.Subject =opvssales_sub\n#mail.Body = 'Hi Team '\nmail.HTMLBody = htmlbody #this field is optional\n\n# To attach a file to the email (optional):\nattachment1  = file_path_master\n#attachment2  = kl_purch_lines\n#attachment3  = tng_purch_lines\n\nmail.Attachments.Add(attachment1)\n#mail.Attachments.Add(attachment2)\n#mail.Attachments.Add(attachment3)\n\nmail.Send()\n"

In [457]:
to_mail = 'karnatakaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com;Keralaops.asterpharmcy@alfaonepharmacy.onmicrosoft.com;Telanganaops.asterpharmacy@alfaonepharmacy.onmicrosoft.com'
cc_mail = 'inventory.asterpharmacy@alfaonepharmacy.onmicrosoft.com;mohammed.arfath@asterpharmacy.in;guruvayur.maharana@asterpharmacy.in;harsha.g@asterpharmacy.in;mudassar.mushtaq@asterpharmacy.in;madhu.n@hindustanpharma.co.in;rajesh.d@asterpharmacy.in;rahul.m@asterpharmacy.in;tom.saji@asterpharmacy.in'

In [458]:
msg = MIMEMultipart()
msg['Subject'] = opvssales_sub
msg.attach(MIMEText(htmlbody, 'html'))

filename = file_path_master

with open(filename, 'rb') as f:
    file = MIMEApplication(
        f.read(),
        name=os.path.basename(filename)
    )
    file['Content-Disposition'] = f'attachment; filename="{os.path.basename(filename)}"'

msg.attach(file)

to = [to_mail]
cc = [cc_mail]  # Replace with the CC email address

# Add CC recipients
msg['To'] = ', '.join(to)
msg['Cc'] = ', '.join(cc)

smtp.sendmail(from_addr="inventory.ho@asterpharmacy.in", to_addrs=to + cc, msg=msg.as_string())
smtp.quit()

(221, b'2.0.0 Service closing transmission channel')